In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
from midas import m
from datetime import datetime, date
import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/VHI01_IST01_Infinity_02_47PM_udpate_1015024"

# Add 47 gene promoter methylation in to the report VHI01_IST01_Infinity_02 

## Data preocessing

In [59]:
# Readin data

df_VHI01_IST01_02batch = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse/20241006_GuardantHealthIST_IST01_Batch2_93Samples_InfinityReport.csv", header = 1) 

print("Total sample size of VHI01_IST01_Infinity_02 Infinity batch is " + str(df_VHI01_IST01_02batch.shape[0]) + "\n") 
print("Total unique sample size of VHI01_IST01_Infinity_02 Infinity batch is " + str(df_VHI01_IST01_02batch.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of VHI01_IST01_Infinity_02 Infinity batch is " + str(df_VHI01_IST01_02batch.Patient_ID.nunique()) + "\n")

df_VHI01_IST01_02batch.head()

Total sample size of VHI01_IST01_Infinity_02 Infinity batch is 12878

Total unique sample size of VHI01_IST01_Infinity_02 Infinity batch is 93

Total unique patient sample size of VHI01_IST01_Infinity_02 Infinity batch is 65



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,TAF1L,9,32631117.0,1,Q1487H,C>G,c.4461G>C,NM_153809.2,51.38,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2165.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
1,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,LRP5,11,68174158.0,9,H656H,C>T,c.1968C>T,NM_002335.4,51.32,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs144847027,Benign,196236.0,NaN,NaN,2052.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
2,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,LZTR1,22,21348422.0,14,P493P,C>T,c.1479C>T,NM_006767.4,51.27,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2065.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
3,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,CARD11,7,2977646.0,8,D346D,G>A,c.1038C>T,NM_032415.7,50.45,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772120748,NaN,NaN,NaN,NaN,1688.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
4,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,FYN,6,112041087.0,4,A56A,G>A,c.168C>T,NM_002037.5,50.23,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs139870258,NaN,NaN,NaN,NaN,2037.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable


In [3]:
# Remove duplication
df_VHI01_IST01_02batch_dedup = df_VHI01_IST01_02batch.drop_duplicates(subset = "GHSampleID")
df_VHI01_IST01_02batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,TAF1L,9,32631117.0,1,Q1487H,C>G,c.4461G>C,NM_153809.2,51.38,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2165.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
71,mCRPC_BIOBANK,GU2385,A1129766,B00771055,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,RB1,13,48947629.0,12,NaN,G>T,c.1215+1G>T,NM_000321.3,53.05,splice_donor_variant,germline,splice_donor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57306748,NaN,NaN,NaN,deleterious,biallelic,1580.0,53.05,NaN,0.408455,Detected,0.576649,NaN,Not evaluable,Not Detected,502.24,2.5,0.0,08/22/2024,08/19/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
231,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,AMER1,X,63412377.0,2,A264T,C>T,c.790G>A,NM_152424.4,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150929706,not_provided,137238.0,NaN,NaN,2520.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
300,mCRPC_BIOBANK,GU2388,A1129768,B00771038,PRO152,Progression,1,SUCCESS,NaN,SNV,NaN,RANBP2,2,109381208.0,20,Q1405E,C>G,c.4213C>G,NM_006267.5,52.02,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV51705648,rs375021752,NaN,NaN,NaN,NaN,2438.0,8.95,NaN,0.044762,Detected,0.081823,6.62,Low,Not Detected,13.49,2.5,0.0,08/22/2024,08/22/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
419,mCRPC_BIOBANK,170-50-020,A1129769,B00771142,PRO040,Baseline,1,SUCCESS,NaN,SNV,NaN,ATRX,X,76918900.0,12,K1364T,T>G,c.4091A>C,NM_000489.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,979.0,8.83,NaN,0.046198,Detected,0.045741,7.56,Low,Not Detected,14.02,2.0,0.0,08/22/2024,07/27/2020,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable


In [5]:
# Pull "runid" from ghdb
df_VHI01_IST01_02batch_dedup_1 = df_VHI01_IST01_02batch_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_VHI01_IST01_02batch_dedup_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_VHI01_IST01_02batch_dedup_1 = df_VHI01_IST01_02batch_dedup_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_VHI01_IST01_02batch_dedup_1 = df_VHI01_IST01_02batch_dedup_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_VHI01_IST01_02batch_dedup_1 = df_VHI01_IST01_02batch_dedup_1.drop(columns='run_date')

print("Total sample size of VHI01_IST01_02batch Infinity batch after merging runid is " + str(df_VHI01_IST01_02batch_dedup_1.shape[0])) 
df_VHI01_IST01_02batch_dedup_1.head()


Total sample size of VHI01_IST01_02batch Infinity batch after merging runid is 93


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
2,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,AMER1,X,63412377.0,2,A264T,C>T,c.790G>A,NM_152424.4,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150929706,not_provided,137238.0,NaN,NaN,2520.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC
3,mCRPC_BIOBANK,GU2388,A1129768,B00771038,PRO152,Progression,1,SUCCESS,NaN,SNV,NaN,RANBP2,2,109381208.0,20,Q1405E,C>G,c.4213C>G,NM_006267.5,52.02,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV51705648,rs375021752,NaN,NaN,NaN,NaN,2438.0,8.95,NaN,0.044762,Detected,0.081823,6.62,Low,Not Detected,13.49,2.5,0.0,08/22/2024,08/22/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC
0,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,TAF1L,9,32631117.0,1,Q1487H,C>G,c.4461G>C,NM_153809.2,51.38,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2165.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC
1,mCRPC_BIOBANK,GU2385,A1129766,B00771055,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,RB1,13,48947629.0,12,NaN,G>T,c.1215+1G>T,NM_000321.3,53.05,splice_donor_variant,germline,splice_donor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57306748,NaN,NaN,NaN,deleterious,biallelic,1580.0,53.05,NaN,0.408455,Detected,0.576649,NaN,Not evaluable,Not Detected,502.24,2.5,0.0,08/22/2024,08/19/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC
49,mCRPC_BIOBANK,GU2442,A1129817,B00771106,PRO131,Progression,1,SUCCESS,NaN,SNV,NaN,KLF4,9,110250098.0,3,V193L,C>G,c.577G>C,NM_001314052.2,54.11,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs751553808,NaN,NaN,NaN,NaN,3232.0,4.61,NaN,0.044537,Detected,0.033290,19.50,Low,Not Detected,28.03,2.5,0.0,08/22/2024,09/13/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02019_0190_AHKNVNDSXC


In [6]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        
        # Check for the first folder that actually exists
        primary_fc_dir = ''
        for folder in folders:
            if os.path.exists(folder):
                primary_fc_dir = folder
                break
        
        # If a valid folder is found
        if primary_fc_dir:
            # Check if there is a 'sirius' folder under the primary fc_dir (for Sirius 1.5 folder structure change)
            if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
            df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_VHI01_IST01_02batch_dedup_2, df_no_found = add_fc_dir_column(df_VHI01_IST01_02batch_dedup_1)

print("Total sample size of VHI01_IST01_Infinity_02 Infinity batch with fc_dir is " + str(df_VHI01_IST01_02batch_dedup_2.shape[0]) + "\n") 
print("Total sample size of VHI01_IST01_Infinity_02 Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_VHI01_IST01_02batch_dedup_2.head()



Total sample size of VHI01_IST01_Infinity_02 Infinity batch with fc_dir is 93

Total sample size of VHI01_IST01_Infinity_02 Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
2,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,AMER1,X,63412377.0,2,A264T,C>T,c.790G>A,NM_152424.4,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150929706,not_provided,137238.0,NaN,NaN,2520.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC
3,mCRPC_BIOBANK,GU2388,A1129768,B00771038,PRO152,Progression,1,SUCCESS,NaN,SNV,NaN,RANBP2,2,109381208.0,20,Q1405E,C>G,c.4213C>G,NM_006267.5,52.02,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV51705648,rs375021752,NaN,NaN,NaN,NaN,2438.0,8.95,NaN,0.044762,Detected,0.081823,6.62,Low,Not Detected,13.49,2.5,0.0,08/22/2024,08/22/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC
0,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,TAF1L,9,32631117.0,1,Q1487H,C>G,c.4461G>C,NM_153809.2,51.38,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2165.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not Detected,15.13,2.5,0.0,08/22/2024,09/09/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC
1,mCRPC_BIOBANK,GU2385,A1129766,B00771055,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,RB1,13,48947629.0,12,NaN,G>T,c.1215+1G>T,NM_000321.3,53.05,splice_donor_variant,germline,splice_donor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57306748,NaN,NaN,NaN,deleterious,biallelic,1580.0,53.05,NaN,0.408455,Detected,0.576649,NaN,Not evaluable,Not Detected,502.24,2.5,0.0,08/22/2024,08/19/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC
49,mCRPC_BIOBANK,GU2442,A1129817,B00771106,PRO131,Progression,1,SUCCESS,NaN,SNV,NaN,KLF4,9,110250098.0,3,V193L,C>G,c.577G>C,NM_001314052.2,54.11,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs751553808,NaN,NaN,NaN,NaN,3232.0,4.61,NaN,0.044537,Detected,0.033290,19.50,Low,Not Detected,28.03,2.5,0.0,08/22/2024,09/13/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02019_0190_AHKNVNDSXC,/ghsfa/ivd/flowcentral/240921_A02019_0190_AHKNVNDSXC


## Generate Pancancer S3 MB caller and Single region caller outputs for this SOW
#### Notes:
    1. No cancer type provided
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [119]:
# Copying the dataframe
df_VHI01_IST01_02batch_dedup_3 = df_VHI01_IST01_02batch_dedup_2.copy()

# List of target genes
Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
                       "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
                       "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
                       "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
                       "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# Initialize columns for each gene in the dataframe
for gene in Target_PM_gene_list:
    df_VHI01_IST01_02batch_dedup_3[f"{gene}_single_region_call_Sirius1_5"] = np.nan

problematic_rows = []  # List to store indices of problematic rows

# Loop through the rows of the dataframe
for index, crc_row in df_VHI01_IST01_02batch_dedup_3.iterrows():
    fc_dir = crc_row["fc_dir"]
    run_sample_id = crc_row["GHSampleID"]

    # Define the file path for the single region report
    single_region_file = f"{fc_dir}/sirius/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

    try:
        # Read the single region call file
        single_df = pd.read_table(single_region_file)

        if not single_df.empty:
            # Loop through the target gene list and assign the call values to the corresponding columns
            for gene in Target_PM_gene_list:
                gene_df = single_df[single_df['gene'] == gene]
                if not gene_df.empty:
                    df_VHI01_IST01_02batch_dedup_3.at[index, f"{gene}_single_region_call_Sirius1_5"] = gene_df.iloc[0]['call']

        else:
            # Add the index to the list if the dataframe is empty
            problematic_rows.append(index) 

    except Exception as e:
        # Add the index to the list if there is any exception
        problematic_rows.append(index)  
        print(f"Error processing file {single_region_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_VHI01_IST01_02batch_dedup_3.loc[problematic_rows]

print(f"Total sample size of VHI01_IST01_Infinity_02 Infinity batch with NO output is {df_problematic.shape[0]}\n")

# Show the updated dataframe with the new columns
df_VHI01_IST01_02batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity_02 Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call_Sirius1_5,ARID1A_single_region_call_Sirius1_5,ARID1B_single_region_call_Sirius1_5,ATM_single_region_call_Sirius1_5,ATR_single_region_call_Sirius1_5,B2M_single_region_call_Sirius1_5,BRCA1_single_region_call_Sirius1_5,BRCA2_single_region_call_Sirius1_5,BRIP1_single_region_call_Sirius1_5,CDH1_single_region_call_Sirius1_5,CDK12_single_region_call_Sirius1_5,CDKN1B_single_region_call_Sirius1_5,CDKN2A_single_region_call_Sirius1_5,CHEK1_single_region_call_Sirius1_5,CHEK2_single_region_call_Sirius1_5,FANCA_single_region_call_Sirius1_5,FANCE_single_region_call_Sirius1_5,FANCF_single_region_call_Sirius1_5,FANCG_single_region_call_Sirius1_5,FANCL_single_region_call_Sirius1_5,KEAP1_single_region_call_Sirius1_5,MGMT_single_region_call_Sirius1_5,MLH1_single_region_call_Sirius1_5,MSH2_single_region_call_Sirius1_5,MSH6_single_region_call_Sirius1_5,MTAP_single_region_call_Sirius1_5,MUTYH_single_region_call_Sirius1_5,NF1_single_region_call_Sirius1_5,NF2_single_region_call_Sirius1_5,PALB2_single_region_call_Sirius1_5,PBRM1_single_region_call_Sirius1_5,PMS2_single_region_call_Sirius1_5,POLE_single_region_call_Sirius1_5,PTEN_single_region_call_Sirius1_5,RAD50_single_region_call_Sirius1_5,RAD51_single_region_call_Sirius1_5,RAD51C_single_region_call_Sirius1_5,RAD51D_single_region_call_Sirius1_5,RB1_single_region_call_Sirius1_5,SDHA_single_region_call_Sirius1_5,SDHB_single_region_call_Sirius1_5,SDHC_single_region_call_Sirius1_5,SDHD_single_region_call_Sirius1_5,SMAD4_single_region_call_Sirius1_5,STK11_single_region_call_Sirius1_5,TP53_single_region_call_Sirius1_5,TSC2_single_region_call_Sirius1_5
2,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,AMER1,X,63412377.0,2,A264T,C>T,c.790G>A,NM_152424.4,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150929706,not_provided,137238.0,NaN,NaN,2520.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,mCRPC_BIOBANK,GU2388,A1129768,B00771038,PRO152,Progression,1,SUCCESS,NaN,SNV,NaN,RANBP2,2,109381208.0,20,Q1405E,C>G,c.4213C>G,NM_006267.5,52.02,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV51705648,rs375021752,NaN,NaN,NaN,NaN,2438.0,8.95,NaN,0.044762,Detected,0.081823,6.62,Low,Not Detected,13.49,2.5,0.0,08/22/2024,08/22/2022,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable,240921_A02056_0234_AHKTYHDSXC,/ghsfa/ivd/flowcentral/240921_A02056_0234_AHKTYHDSXC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,mCRPC_BIOBANK,170-50-062,A1129765,B00771054,PRO246,Baseline,1,SUCCESS,NaN,SNV,NaN,TAF1L,9,32631117.0,1,Q1487H,C>G,c.4461G>C,NM_153809.2,51.38,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2165.0,0.49,NaN,0.272128,Detected,0.010835,7.09,Low,Not 

In [120]:
# Retrive the 47 PM gene binary call result

df_VHI01_IST01_02batch_dedup_4 = df_VHI01_IST01_02batch_dedup_3.copy()

# List of 47 LDT reportable PM genes
Target_PM_gene_list = [
    "APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", "CDK12", 
    "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", "FANCL", "KEAP1", 
    "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", "PALB2", "PBRM1", "PMS2", 
    "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", "RB1", "SDHA", "SDHB", "SDHC", 
    "SDHD", "SMAD4", "STK11", "TP53", "TSC2"
]

# Process each gene for each sample
new_rows = []
for index, row in df_VHI01_IST01_02batch_dedup_4.iterrows():
    GHSampleID = row['GHSampleID']
    for gene in Target_PM_gene_list:
        gene_call = row[f'{gene}_single_region_call_Sirius1_5']
        if pd.notna(gene_call):
            variant_type = 'PromoterMethylationDetected' if gene_call == 1 else 'PromoterMethylationNotDetected'
            new_rows.append({'GHSampleID': GHSampleID, 'Variant_type': variant_type, 'Gene': gene})

# Append new rows to the dataframe
df_47_PM_gene = pd.DataFrame(new_rows)
df_47_PM_gene.head()


,GHSampleID,Variant_type,Gene
0,B00771036,PromoterMethylationNotDetected,APC
1,B00771036,PromoterMethylationNotDetected,ARID1A
2,B00771036,PromoterMethylationNotDetected,ARID1B
3,B00771036,PromoterMethylationNotDetected,ATM
4,B00771036,PromoterMethylationNotDetected,ATR


In [129]:
# Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
df_promoter_methylation = df_VHI01_IST01_02batch[df_VHI01_IST01_02batch['Variant_type'] == 'PromoterMethylation'].copy()

# Step 2: Remove these rows from the original dataframe to avoid duplication later
df_non_promoter_methylation = df_VHI01_IST01_02batch[df_VHI01_IST01_02batch['Variant_type'] != 'PromoterMethylation'].copy()

# Step 3: Include the 'Study_ID', 'Variant_type', 'Gene' columns and other necessary columns
df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
                                                           'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
                                                           'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
                                                           'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
                                                           'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
                                                           'Variant_type', 'Gene']].copy()

# Step 4: Merge with df_47_PM_gene using a left join, keeping GHSampleID match and suffixes to avoid conflicts
df_promoter_methylation_updated = df_promoter_methylation_updated.merge(df_47_PM_gene, on='GHSampleID', how='left', suffixes=('', '_PM'))

# Step 5: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# Step 6: Drop the temporary columns generated during the merge
df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# Step 7: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
all_ghsampleid = df_VHI01_IST01_02batch['GHSampleID'].unique()
missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# Step 8: For these missing GHSampleIDs, manually create the "PromoterMethylation" rows using df_47_PM_gene
df_missing_pm_rows = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(missing_ghsampleid)].copy()

# Add the necessary columns with NaN or default values for the missing GHSampleID samples
for col in ['Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
            'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
            'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
            'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
            'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name']:
    df_missing_pm_rows[col] = None  # Assign default values or NaN for these columns

# Step 9: Append the missing rows to the updated promoter methylation rows
df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# Step 10: Append the updated rows back to the original dataframe while keeping their positions
df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# Step 11: Sort the dataframe by GHSampleID and place rows accordingly
df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# Step 12: Ensure there are no duplicates
df_ori_with_47pm.drop_duplicates(inplace=True)

df_ori_with_47pm.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
145,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,AMER1,X,63412377.0,2,A264T,C>T,c.790G>A,NM_152424.4,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150929706,not_provided,137238.0,NaN,NaN,2520.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
146,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,GATA1,X,48649689.0,2,A58V,C>T,c.173C>T,NM_002049.4,99.96,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV64963758,rs782299679,NaN,NaN,NaN,NaN,2397.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
147,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,ZFHX3,16,72821592.0,10,S3528L,G>A,c.10583C>T,NM_006885.4,60.97,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV51731993,rs781184186,NaN,NaN,NaN,NaN,350.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
148,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,JAK1,1,65307190.0,18,N833S,T>C,c.2498A>G,NM_002227.4,50.65,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs187043211,not_provided,138283.0,NaN,NaN,5596.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable
149,mCRPC_BIOBANK,GU1297,A1129767,B00771036,PRO124,Baseline,1,SUCCESS,NaN,SNV,NaN,GAS6,13,114530064.0,12,T461M,G>A,c.1382C>T,NM_000820.4,50.42,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs202234400,NaN,NaN,NaN,NaN,4973.0,0.92,NaN,0.255697,Detected,0.005206,10.53,Low,Not Detected,220.17,3.0,0.0,08/22/2024,01/08/2021,10/06/2024,Metastatic prostate cancer,Not Applicable,Not Applicable


In [131]:
# Check the result 
print(df_VHI01_IST01_02batch.shape)
print(df_47_PM_gene.shape)
df_non_promoter_methylation = df_VHI01_IST01_02batch[df_VHI01_IST01_02batch['Variant_type'] != 'PromoterMethylation'].copy()
print(df_non_promoter_methylation.shape)
print(df_ori_with_47pm.shape)

# Save the temporary data
df_ori_with_47pm.to_csv(f"{work_dir}/df_ori_with_47pm.csv", index = False)

(12878, 56)
(4371, 3)
(8911, 56)
(13282, 56)


In [134]:
# Additional dataframe updates based on Rachael's request
# Drop two columns
df_ori_with_47pm_removed_2col = df_ori_with_47pm.drop(columns=['HRD_score', 'Alleletype'])

# Drop IO Allele type info
df_final = df_ori_with_47pm_removed_2col[df_ori_with_47pm_removed_2col['Variant_type'] != "AlleleType"]

# Save the data
df_final.to_csv(f"{work_dir}/df_final.csv", index = False)
df_final.shape

(10399, 54)

# Add 47 gene promoter methylation in to the report VHI01_IST01_Infinity_01

## Data preocessing

In [4]:
# Readin data

df_VHI01_IST01_01batch = pd.read_csv(f"/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse/20240207_GuardantHealthIST_IST01_54Samples_InfinityReport_HRDscore.csv", header = 1) 

print("Total sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.shape[0]) + "\n") 
print("Total unique sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.Patient_ID.nunique()) + "\n")

df_VHI01_IST01_01batch.head()

Total sample size of df_VHI01_IST01_01batch Infinity batch is 6408

Total unique sample size of df_VHI01_IST01_01batch Infinity batch is 54

Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is 40



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9.0,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.6524,Detected,0.17404,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
1,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187539925.0,10.0,I2605I,G>A,c.7815C>T,NM_005245.4,52.40,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71673524,rs750802295,NaN,NaN,NaN,NaN,4970.0,18.93,NaN,0.6524,Detected,0.17404,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
2,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,PIM1,6,37138190.0,1.0,S38T,T>A,c.112T>A,NM_001243186.1,51.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1042528446,NaN,NaN,NaN,NaN,2153.0,18.93,NaN,0.6524,Detected,0.17404,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
3,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187531074.0,15.0,D3317N,C>T,c.9949G>A,NM_005245.4,50.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71675068,rs199989457,Likely_benign,709287.0,NaN,NaN,4731.0,18.93,NaN,0.6524,Detected,0.17404,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
4,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,ABRAXAS1,4,84383761.0,9.0,D364G,T>C,c.1091A>G,NM_139076.3,49.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV55029548,rs142910445,NaN,NaN,NaN,NaN,4748.0,18.93,NaN,0.6524,Detected,0.17404,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable


In [18]:
df_VHI01_IST01_01batch.Customer_SampleID.value_counts()

GU1406    316
GU1332    305
GU1640    304
GU1492    275
GU1236    196
GU1168    191
GU0299    184
GU1194    184
GU0498    179
GU2092    174
GU0646    164
GU1556    158
GU2173    155
GU1641    153
GU2126    149
GU0970    148
GU1133    146
GU1049    143
GU1585    142
GU0422    139
GU0680    126
GU0078    124
GU0673    124
GU1670    123
GU1154    123
GU1770    121
GU0473    118
GU0685    114
GU0810    105
GU1526    104
GU2162    101
GU1674    100
GU1430     90
GU1218     85
GU0134     82
GU1157     77
GU2002     76
GU1886     75
GU0890     74
GU1676     66
GU1925     65
GU1329     63
GU1393     59
GU1683     56
GU0866     55
GU1002     54
GU1162     52
GU0410     49
GU0414     48
GU1809     46
GU1622     45
GU1217      1
GU0426      1
GU0377      1
Name: Customer_SampleID, dtype: int64

In [17]:
# Address the sample mislabel issue from the PI side
# We corrected the mistake in the discrepancy form that Isart sent to us. However, I noticed that the results you shared still included the sample PRO151_GU1556, which is actually PRO019_GU1566.
# mask_to_change = (df_VHI01_IST01_01batch['Customer_SampleID'] == 'GU1566') & (df_VHI01_IST01_01batch['Patient_ID'] == 'PRO151') & (df_VHI01_IST01_01batch['Visit_name'] == 'Progression')
mask_to_change = (df_VHI01_IST01_01batch['Customer_SampleID'] == 'GU1556') # The number they provided is not right!!!

df_test = df_VHI01_IST01_01batch[mask_to_change]
df_test

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name


In [3]:
# Remove duplication
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch.drop_duplicates(subset = "GHSampleID")
df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9.0,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.6524,Detected,0.174040,9.45,Low,Not Detected,237.12,2.5,NaN,11/9/23,10/15/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
126,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,1,SUCCESS,Warning: Low Diversity,SNV,NaN,FGFR3,4,1806663.0,10.0,P460L,C>T,c.1379C>T,NM_000142.5,74.62,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs764488842,NaN,NaN,NaN,NaN,534.0,2.94,NaN,NaN,Detected,0.258176,NaN,Not evaluable,Not Detected,45.50,2.5,NaN,11/9/23,10/21/21,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
247,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49.0,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,13.59,NaN,0.2848,Detected,0.306871,7.56,Low,Not Detected,74.88,2.5,NaN,11/9/23,9/9/20,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
393,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49.0,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.7071,Detected,0.919525,5.67,Low,Not Detected,474.24,3.5,NaN,11/9/23,1/25/21,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
698,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10.0,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.9445,Detected,0.791199,20.80,High,Not Detected,481.52,3.0,NaN,11/9/23,5/4/21,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable


In [5]:
df_VHI01_IST01_01batch_dedup.Cancertype.value_counts()

Series([], Name: Cancertype, dtype: int64)

In [6]:
# Pull "runid" from ghdb
df_VHI01_IST01_01batch_dedup_1 = df_VHI01_IST01_01batch_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_VHI01_IST01_01batch_dedup_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_VHI01_IST01_01batch_dedup_1 = df_VHI01_IST01_01batch_dedup_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_VHI01_IST01_01batch_dedup_1 = df_VHI01_IST01_01batch_dedup_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_VHI01_IST01_01batch_dedup_1 = df_VHI01_IST01_01batch_dedup_1.drop(columns='run_date')

print("Total sample size of VHI01_IST01_01batch_dedup_1 Infinity batch after merging runid is " + str(df_VHI01_IST01_01batch_dedup_1.shape[0])) 
df_VHI01_IST01_01batch_dedup_1.head()


Total sample size of VHI01_IST01_01batch_dedup_1 Infinity batch after merging runid is 54


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
6,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2.0,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.3877,Detected,0.288620,17.01,Low,Not Detected,83.20,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7
7,mCRPC_BIOBANK,GU0685,A0914048,B00586982,PRO046,Baseline,1,SUCCESS,NaN,SNV,NaN,PTPRT,20,40733305.0,26.0,N1167K,G>T,c.3501C>A,NM_133170.4,52.32,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV61980909,rs201162919,NaN,NaN,NaN,NaN,5469.0,5.98,NaN,0.0661,Detected,0.052270,14.18,Low,Not Detected,30.78,1.5,NaN,11/9/23,10/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7
51,mCRPC_BIOBANK,GU1217,A0914045,B00586984,PRO087,Progression,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.84,3.0,NaN,11/9/23,11/9/20,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7
5,mCRPC_BIOBANK,GU1218,A0914046,B00586985,PRO120,Baseline,1,SUCCESS,NaN,SNV,NaN,VHL,3,10183714.0,1.0,P61P,C>G,c.183C>G,NM_000551.4,51.91,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV56546597,rs63650860,Conflicting_interpretations_of_pathogenicity,139663.0,NaN,NaN,895.0,1.32,NaN,0.0554,Detected,0.013979,7.09,Low,Not Detected,9.62,3.0,NaN,11/9/23,11/9/20,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7
3,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49.0,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.7071,Detected,0.919525,5.67,Low,Not Detected,474.24,3.5,NaN,11/9/23,1/25/21,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7


In [7]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        
        # Check for the first folder that actually exists
        primary_fc_dir = ''
        for folder in folders:
            if os.path.exists(folder):
                primary_fc_dir = folder
                break
        
        # If a valid folder is found
        if primary_fc_dir:
            # Check if there is a 'sirius' folder under the primary fc_dir (for Sirius 1.5 folder structure change)
            if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
            df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_VHI01_IST01_01batch_dedup_2, df_no_found = add_fc_dir_column(df_VHI01_IST01_01batch_dedup_1)

print("Total sample size of VHI01_IST01_Infinity_01 Infinity batch with fc_dir is " + str(df_VHI01_IST01_01batch_dedup_2.shape[0]) + "\n") 
print("Total sample size of VHI01_IST01_Infinity_01 Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_VHI01_IST01_01batch_dedup_2.head()



Total sample size of VHI01_IST01_Infinity_01 Infinity batch with fc_dir is 54

Total sample size of VHI01_IST01_Infinity_01 Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
6,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2.0,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.3877,Detected,0.288620,17.01,Low,Not Detected,83.20,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658
7,mCRPC_BIOBANK,GU0685,A0914048,B00586982,PRO046,Baseline,1,SUCCESS,NaN,SNV,NaN,PTPRT,20,40733305.0,26.0,N1167K,G>T,c.3501C>A,NM_133170.4,52.32,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV61980909,rs201162919,NaN,NaN,NaN,NaN,5469.0,5.98,NaN,0.0661,Detected,0.052270,14.18,Low,Not Detected,30.78,1.5,NaN,11/9/23,10/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658
51,mCRPC_BIOBANK,GU1217,A0914045,B00586984,PRO087,Progression,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.84,3.0,NaN,11/9/23,11/9/20,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658
5,mCRPC_BIOBANK,GU1218,A0914046,B00586985,PRO120,Baseline,1,SUCCESS,NaN,SNV,NaN,VHL,3,10183714.0,1.0,P61P,C>G,c.183C>G,NM_000551.4,51.91,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV56546597,rs63650860,Conflicting_interpretations_of_pathogenicity,139663.0,NaN,NaN,895.0,1.32,NaN,0.0554,Detected,0.013979,7.09,Low,Not Detected,9.62,3.0,NaN,11/9/23,11/9/20,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658
3,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49.0,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.7071,Detected,0.919525,5.67,Low,Not Detected,474.24,3.5,NaN,11/9/23,1/25/21,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658


## Generate Pancancer S3 MB caller and Single region caller outputs for this SOW
#### Notes:
    1. No cancer type provided
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [10]:
# Copying the dataframe
df_VHI01_IST01_01batch_dedup_3 = df_VHI01_IST01_01batch_dedup_2.copy()

# List of target genes
Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
                       "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
                       "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
                       "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
                       "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# Initialize columns for each gene in the dataframe
for gene in Target_PM_gene_list:
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call_Sirius1_5"] = np.nan

problematic_rows = []  # List to store indices of problematic rows

# Loop through the rows of the dataframe
for index, crc_row in df_VHI01_IST01_01batch_dedup_3.iterrows():
    fc_dir = crc_row["fc_dir"]
    run_sample_id = crc_row["GHSampleID"]

    # Define the file path for the single region report
    single_region_file = f"{fc_dir}/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

    try:
        # Read the single region call file
        single_df = pd.read_table(single_region_file)

        if not single_df.empty:
            # Loop through the target gene list and assign the call values to the corresponding columns
            for gene in Target_PM_gene_list:
                gene_df = single_df[single_df['gene'] == gene]
                if not gene_df.empty:
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call_Sirius1_5"] = gene_df.iloc[0]['call']

        else:
            # Add the index to the list if the dataframe is empty
            problematic_rows.append(index) 

    except Exception as e:
        # Add the index to the list if there is any exception
        problematic_rows.append(index)  
        print(f"Error processing file {single_region_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_VHI01_IST01_01batch_dedup_3.loc[problematic_rows]

print(f"Total sample size of VHI01_IST01_Infinity_01 Infinity batch with NO output is {df_problematic.shape[0]}\n")

# Show the updated dataframe with the new columns
df_VHI01_IST01_01batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity_01 Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call_Sirius1_5,ARID1A_single_region_call_Sirius1_5,ARID1B_single_region_call_Sirius1_5,ATM_single_region_call_Sirius1_5,ATR_single_region_call_Sirius1_5,B2M_single_region_call_Sirius1_5,BRCA1_single_region_call_Sirius1_5,BRCA2_single_region_call_Sirius1_5,BRIP1_single_region_call_Sirius1_5,CDH1_single_region_call_Sirius1_5,CDK12_single_region_call_Sirius1_5,CDKN1B_single_region_call_Sirius1_5,CDKN2A_single_region_call_Sirius1_5,CHEK1_single_region_call_Sirius1_5,CHEK2_single_region_call_Sirius1_5,FANCA_single_region_call_Sirius1_5,FANCE_single_region_call_Sirius1_5,FANCF_single_region_call_Sirius1_5,FANCG_single_region_call_Sirius1_5,FANCL_single_region_call_Sirius1_5,KEAP1_single_region_call_Sirius1_5,MGMT_single_region_call_Sirius1_5,MLH1_single_region_call_Sirius1_5,MSH2_single_region_call_Sirius1_5,MSH6_single_region_call_Sirius1_5,MTAP_single_region_call_Sirius1_5,MUTYH_single_region_call_Sirius1_5,NF1_single_region_call_Sirius1_5,NF2_single_region_call_Sirius1_5,PALB2_single_region_call_Sirius1_5,PBRM1_single_region_call_Sirius1_5,PMS2_single_region_call_Sirius1_5,POLE_single_region_call_Sirius1_5,PTEN_single_region_call_Sirius1_5,RAD50_single_region_call_Sirius1_5,RAD51_single_region_call_Sirius1_5,RAD51C_single_region_call_Sirius1_5,RAD51D_single_region_call_Sirius1_5,RB1_single_region_call_Sirius1_5,SDHA_single_region_call_Sirius1_5,SDHB_single_region_call_Sirius1_5,SDHC_single_region_call_Sirius1_5,SDHD_single_region_call_Sirius1_5,SMAD4_single_region_call_Sirius1_5,STK11_single_region_call_Sirius1_5,TP53_single_region_call_Sirius1_5,TSC2_single_region_call_Sirius1_5
6,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2.0,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.3877,Detected,0.288620,17.01,Low,Not Detected,83.20,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,mCRPC_BIOBANK,GU0685,A0914048,B00586982,PRO046,Baseline,1,SUCCESS,NaN,SNV,NaN,PTPRT,20,40733305.0,26.0,N1167K,G>T,c.3501C>A,NM_133170.4,52.32,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV61980909,rs201162919,NaN,NaN,NaN,NaN,5469.0,5.98,NaN,0.0661,Detected,0.052270,14.18,Low,Not Detected,30.78,1.5,NaN,11/9/23,10/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghsfa/ivd/flowcentral/240125_A01361_0592_AHMMTVDSX7.d9ab8c28-bb91-4cc3-b687-0a99abb170ba.20240127130658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,mCRPC_BIOBANK,GU1217,A0914045,B00586984,PRO087,Progression,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
# Retrive the 47 PM gene binary call result

df_VHI01_IST01_01batch_dedup_4 = df_VHI01_IST01_01batch_dedup_3.copy()

# List of 47 LDT reportable PM genes
Target_PM_gene_list = [
    "APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", "CDK12", 
    "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", "FANCL", "KEAP1", 
    "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", "PALB2", "PBRM1", "PMS2", 
    "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", "RB1", "SDHA", "SDHB", "SDHC", 
    "SDHD", "SMAD4", "STK11", "TP53", "TSC2"
]

# Process each gene for each sample
new_rows = []
for index, row in df_VHI01_IST01_01batch_dedup_4.iterrows():
    GHSampleID = row['GHSampleID']
    for gene in Target_PM_gene_list:
        gene_call = row[f'{gene}_single_region_call_Sirius1_5']
        if pd.notna(gene_call):
            variant_type = 'PromoterMethylationDetected' if gene_call == 1 else 'PromoterMethylationNotDetected'
            new_rows.append({'GHSampleID': GHSampleID, 'Variant_type': variant_type, 'Gene': gene})

# Append new rows to the dataframe
df_47_PM_gene = pd.DataFrame(new_rows)
df_47_PM_gene.head()


,GHSampleID,Variant_type,Gene
0,B00586981,PromoterMethylationNotDetected,APC
1,B00586981,PromoterMethylationNotDetected,ARID1A
2,B00586981,PromoterMethylationNotDetected,ARID1B
3,B00586981,PromoterMethylationNotDetected,ATM
4,B00586981,PromoterMethylationNotDetected,ATR


In [12]:
df_47_PM_gene.shape

(2538, 3)

In [13]:
# Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
df_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] == 'PromoterMethylation'].copy()

# Step 2: Remove these rows from the original dataframe to avoid duplication later
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()

# Step 3: Include the 'Study_ID', 'Variant_type', 'Gene' columns and other necessary columns
df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
                                                           'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
                                                           'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
                                                           'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
                                                           'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
                                                           'Variant_type', 'Gene']].copy()

# Step 4: Merge with df_47_PM_gene using a left join, keeping GHSampleID match and suffixes to avoid conflicts
df_promoter_methylation_updated = df_promoter_methylation_updated.merge(df_47_PM_gene, on='GHSampleID', how='left', suffixes=('', '_PM'))

# Step 5: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# Step 6: Drop the temporary columns generated during the merge
df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# Step 7: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
all_ghsampleid = df_VHI01_IST01_01batch['GHSampleID'].unique()
missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# Step 8: For these missing GHSampleIDs, manually create the "PromoterMethylation" rows using df_47_PM_gene
df_missing_pm_rows = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(missing_ghsampleid)].copy()

# Add the necessary columns with NaN or default values for the missing GHSampleID samples
for col in ['Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
            'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
            'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
            'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
            'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name']:
    df_missing_pm_rows[col] = None  # Assign default values or NaN for these columns

# Step 9: Append the missing rows to the updated promoter methylation rows
df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# Step 10: Append the updated rows back to the original dataframe while keeping their positions
df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# Step 11: Sort the dataframe by GHSampleID and place rows accordingly
df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# Step 12: Ensure there are no duplicates
df_ori_with_47pm.drop_duplicates(inplace=True)

df_ori_with_47pm.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
793,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2.0,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.3877,Detected,0.28862,17.01,Low,Not Detected,83.2,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
794,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,TET2,4,106156445.0,3.0,V449A,T>C,c.1346T>C,NM_001127208.3,52.75,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs753698541,NaN,NaN,NaN,NaN,6245.0,20.41,NaN,0.3877,Detected,0.28862,17.01,Low,Not Detected,83.2,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
795,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FAAP100,17,79511105.0,7.0,A784A,G>A,c.2352C>T,NM_025161.6,51.45,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs142183217,NaN,NaN,NaN,NaN,5688.0,20.41,NaN,0.3877,Detected,0.28862,17.01,Low,Not Detected,83.2,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
796,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,LRP5,11,68177472.0,10.0,M728L,A>T,c.2182A>T,NM_002335.4,51.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs149241739,NaN,NaN,NaN,NaN,5526.0,20.41,NaN,0.3877,Detected,0.28862,17.01,Low,Not Detected,83.2,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable
797,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FOXA1,14,38060693.0,2.0,T432T,C>G,c.1296G>C,NM_004496.5,51.06,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375938424,NaN,NaN,NaN,NaN,5745.0,20.41,NaN,0.3877,Detected,0.28862,17.01,Low,Not Detected,83.2,2.5,NaN,11/9/23,9/23/19,2/7/24,NaN,Not Applicable,Not Applicable Not Applicable


In [14]:
# Check the result 
print(df_VHI01_IST01_01batch.shape)
print(df_47_PM_gene.shape)
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()
print(df_non_promoter_methylation.shape)
print(df_ori_with_47pm.shape)

# Save the temporary data
df_ori_with_47pm.to_csv(f"{work_dir}/df_ori_with_47pm_batch01.csv", index = False)

(6408, 56)
(2538, 3)
(4827, 56)
(7365, 56)


In [15]:
# Additional dataframe updates based on Rachael's request
# Drop two columns
df_ori_with_47pm_removed_2col = df_ori_with_47pm.drop(columns=['HRD_score', 'Alleletype'])

# Drop IO Allele type info
df_final = df_ori_with_47pm_removed_2col[df_ori_with_47pm_removed_2col['Variant_type'] != "AlleleType"]

# Save the data
df_final.to_csv(f"{work_dir}/df_final_batch01.csv", index = False)
df_final.shape

(5784, 54)

# Add 47 gene promoter methylation in to the report VHI01_IST01_Infinity (all 147 BIP rerun)

## Data preocessing

In [41]:
# Readin data

df_VHI01_IST01_01batch = pd.read_csv(f"{work_dir}/20241206_GuardantHealthIST_IST01_Batch1_147Samples_InfinityReport.csv", header = 1) 

print("Total sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.shape[0]) + "\n") 
print("Total unique sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.Patient_ID.nunique()) + "\n")

df_VHI01_IST01_01batch.head()

Total sample size of df_VHI01_IST01_01batch Infinity batch is 11815

Total unique sample size of df_VHI01_IST01_01batch Infinity batch is 147

Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is 98



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
1,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187539925.0,10,I2605I,G>A,c.7815C>T,NM_005245.4,52.40,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71673524,rs750802295,NaN,NaN,NaN,NaN,4970.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
2,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,PIM1,6,37138190.0,1,S38T,T>A,c.112T>A,NM_001243186.1,51.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1042528446,NaN,NaN,NaN,NaN,2153.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
3,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187531074.0,15,D3317N,C>T,c.9949G>A,NM_005245.4,50.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71675068,rs199989457,Likely_benign,709287.0,NaN,NaN,4731.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
4,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,ABRAXAS1,4,84383761.0,9,D364G,T>C,c.1091A>G,NM_139076.3,49.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV55029548,rs142910445,NaN,NaN,NaN,NaN,4748.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable


In [42]:
# Address the sample mislabel issue from the PI side
# We corrected the mistake in the discrepancy form that Isart sent to us. However, I noticed that the results you shared still included the sample PRO151_GU1556, which is actually PRO019_GU1566.
# mask_to_change = (df_VHI01_IST01_01batch['Customer_SampleID'] == 'GU1566') & (df_VHI01_IST01_01batch['Patient_ID'] == 'PRO151') & (df_VHI01_IST01_01batch['Visit_name'] == 'Progression')
mask_to_change = (df_VHI01_IST01_01batch['Customer_SampleID'] == 'GU1556') 

df_test = df_VHI01_IST01_01batch[mask_to_change]
df_test.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
3520,mCRPC_BIOBANK,GU1556,A0914082,B00587783,PRO019,Progression,1,SUCCESS,NaN,SNV,NaN,ESR1,6,152419906.0,10,K531K,G>A,c.1593G>A,NM_001122742.2,74.58,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3214.0,51.59,NaN,0.267477,Detected,0.75391,12.29,Low,Not Detected,480.48,2.0,0,11/9/23,6/8/21,12/6/24,NaN,Not Applicable,Not Applicable
3521,mCRPC_BIOBANK,GU1556,A0914082,B00587783,PRO019,Progression,1,SUCCESS,NaN,SNV,NaN,CYP2C19,10,96535209.0,3,R132W,C>T,c.394C>T,NM_000769.4,73.80,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV64907043,rs149590953,NaN,NaN,NaN,NaN,3479.0,51.59,NaN,0.267477,Detected,0.75391,12.29,Low,Not Detected,480.48,2.0,0,11/9/23,6/8/21,12/6/24,NaN,Not Applicable,Not Applicable
3522,mCRPC_BIOBANK,GU1556,A0914082,B00587783,PRO019,Progression,1,SUCCESS,NaN,SNV,NaN,ERCC6,10,50701183.0,8,G601S,C>T,c.1801G>A,NM_000124.4,73.27,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV63389976,rs138758064,NaN,NaN,NaN,NaN,2760.0,51.59,NaN,0.267477,Detected,0.75391,12.29,Low,Not Detected,480.48,2.0,0,11/9/23,6/8/21,12/6/24,NaN,Not Applicable,Not Applicable
3523,mCRPC_BIOBANK,GU1556,A0914082,B00587783,PRO019,Progression,1,SUCCESS,NaN,SNV,NaN,FANCD2,3,10132062.0,37,S1257L,C>T,c.3770C>T,NM_033084.6,57.26,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99811362,rs374940277,NaN,NaN,NaN,NaN,5292.0,51.59,NaN,0.267477,Detected,0.75391,12.29,Low,Not Detected,480.48,2.0,0,11/9/23,6/8/21,12/6/24,NaN,Not Applicable,Not Applicable
3524,mCRPC_BIOBANK,GU1556,A0914082,B00587783,PRO019,Progression,1,SUCCESS,NaN,SNV,NaN,XPC,3,14200206.0,9,R393W,G>A,c.1177C>T,NM_004628.5,57.07,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV104373862,rs121965090,Uncertain_significance,139223.0,NaN,NaN,6226.0,51.59,NaN,0.267477,Detected,0.75391,12.29,Low,Not Detected,480.48,2.0,0,11/9/23,6/8/21,12/6/24,NaN,Not Applicable,Not Applicable


In [43]:
# Remove duplication
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch.drop_duplicates(subset = "GHSampleID")
df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
90,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable
91,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,14.66,NaN,0.137291,Detected,0.369632,NaN,Not evaluable,Not Detected,74.88,2.5,0,11/9/23,9/9/20,12/6/24,NaN,Not Applicable,Not Applicable
210,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.436673,Detected,0.990000,NaN,Not evaluable,Not Detected,474.24,3.5,0,11/9/23,1/25/21,12/6/24,NaN,Not Applicable,Not Applicable
454,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.931165,Detected,0.976160,20.80,High,Not Detected,481.52,3.0,0,11/9/23,5/4/21,12/6/24,NaN,Not Applicable,Not Applicable


In [44]:
# Merge runid based on re-run BART v0 https://docs.google.com/spreadsheets/d/1NMnfaUHrVL-6FfzzoaB2yWBVqTihUCF1a1pHgUg7g-8/edit?gid=0#gid=0
# Readin the file with runid
df_VHI01_IST01_01batch_runid = pd.read_csv(f"{work_dir}/rerun_v0_for_runid.csv", header = 0) 
df_VHI01_IST01_01batch_runid = df_VHI01_IST01_01batch_runid.drop_duplicates(subset = 'run_sample_id')
df_VHI01_IST01_01batch_runid.head()

# Merge the runid back
df_VHI01_IST01_01batch_runid.rename(columns={'run_sample_id': 'GHSampleID'}, inplace=True)
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch_dedup.merge(df_VHI01_IST01_01batch_runid[["GHSampleID", "runid"]], on = "GHSampleID", how = "left")

df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7
1,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7
2,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,14.66,NaN,0.137291,Detected,0.369632,NaN,Not evaluable,Not Detected,74.88,2.5,0,11/9/23,9/9/20,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7
3,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.436673,Detected,0.990000,NaN,Not evaluable,Not Detected,474.24,3.5,0,11/9/23,1/25/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7
4,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.931165,Detected,0.976160,20.80,High,Not Detected,481.52,3.0,0,11/9/23,5/4/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7


In [11]:
def add_fc_dir_column_rerun(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    # fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_all = "/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells"
    # fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        
        # Check for the first folder that actually exists
        primary_fc_dir = ''
        for folder in folders:
            if os.path.exists(folder):
                primary_fc_dir = folder
                break
        
        # If a valid folder is found
        if primary_fc_dir:
            # Check if there is a 'sirius' folder under the primary fc_dir (for Sirius 1.5 folder structure change)
            if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
            df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_VHI01_IST01_01batch_dedup_2, df_no_found = add_fc_dir_column_rerun(df_VHI01_IST01_01batch_dedup)

print("Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is " + str(df_VHI01_IST01_01batch_dedup_2.shape[0]) + "\n") 
print("Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_VHI01_IST01_01batch_dedup_2.head()



Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is 147

Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0
1,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0
2,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,14.66,NaN,0.137291,Detected,0.369632,NaN,Not evaluable,Not Detected,74.88,2.5,0,11/9/23,9/9/20,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0
3,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.436673,Detected,0.990000,NaN,Not evaluable,Not Detected,474.24,3.5,0,11/9/23,1/25/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0
4,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.931165,Detected,0.976160,20.80,High,Not Detected,481.52,3.0,0,11/9/23,5/4/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0


In [ ]:
# # THIS IS ONLY for saving intermediate file purpose! DO NO RUN!
# def add_fc_dir_column_rerun(df):
#     """
#     Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

#     Args:
#         df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
#                            match directory paths from predefined locations.

#     Returns:
#         pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
#                       corresponding to each 'runid'. If no directory is found, the path will be an empty string.
#         pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
#     """
#     # Define the path
#     # fc_dir_all = "/ghsfa/ivd/flowcentral"
#     fc_dir_all = "/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells"
#     # fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
#     # Initialize a list to store rows where no BIP output was found
#     rows_no_found = []
    
#     # Initialize the new column in the DataFrame
#     df['fc_dir'] = ''
    
#     # Iterate over the DataFrame rows
#     for index, row in df.iterrows():
#         runid = row["runid"]
        
#         # Find original folder
#         pattern_ori = f"{fc_dir_all}/*{runid}*"
#         folders = glob.glob(pattern_ori)
        
#         # Check for the first folder that actually exists
#         primary_fc_dir = ''
#         for folder in folders:
#             if os.path.exists(folder):
#                 primary_fc_dir = folder
#                 break
        
#         # If a valid folder is found
#         if primary_fc_dir:
#             # Check if there is a 'sirius' folder under the primary fc_dir (for Sirius 1.5 folder structure change)
#             if os.path.exists(os.path.join(primary_fc_dir, "sirius_1_6_0")):
#                 primary_fc_dir = os.path.join(primary_fc_dir, "sirius_1_6_0")
#             df.loc[index, 'fc_dir'] = primary_fc_dir
#         else:
#             # Search in the alternative directory if no folder found in the first directory
#             pattern_omni = f"{fc_dir_omni}/*{runid}*"
#             folders_omni = glob.glob(pattern_omni)
#             if folders_omni:
#                 df.loc[index, 'fc_dir'] = folders_omni[0]
#             else:
#                 # Append the row to the list if no folder is found in either location
#                 rows_no_found.append(row)
    
#     # Create a DataFrame with rows where folders were not found in any sources
#     df_no_fc_dir = pd.DataFrame(rows_no_found)
    
#     return df, df_no_fc_dir

# # Generate the new dataframe 
# df_VHI01_IST01_01batch_dedup_2, df_no_found = add_fc_dir_column_rerun(df_VHI01_IST01_01batch_dedup)

# print("Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is " + str(df_VHI01_IST01_01batch_dedup_2.shape[0]) + "\n") 
# print("Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

# df_VHI01_IST01_01batch_dedup_2.head()


# # Save the data for cummulative intermediate file
# df_VHI01_IST01_01batch_dedup_2.to_csv(f"{work_dir}/df_VHI01_IST01_01batch_dedup_2_batch1and2_147_samples_with_rerun_fc_dir_02202025.csv", index = False)

Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is 147

Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is 0



## Generate Pancancer S3 MB caller and Single region caller outputs for this SOW
#### Notes:
    1. No cancer type provided
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [ ]:
# # Copying the dataframe
# df_VHI01_IST01_01batch_dedup_3 = df_VHI01_IST01_01batch_dedup_2.copy()

# # List of target genes
# Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
#                        "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
#                        "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
#                        "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
#                        "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# # Initialize columns for each gene in the dataframe
# for gene in Target_PM_gene_list:
#     df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call_Sirius1_6_0"] = np.nan

# problematic_rows = []  # List to store indices of problematic rows

# # Loop through the rows of the dataframe
# for index, crc_row in df_VHI01_IST01_01batch_dedup_3.iterrows():
#     fc_dir = crc_row["fc_dir"]
#     run_sample_id = crc_row["GHSampleID"]

#     # Define the file path for the single region report
#     single_region_file = f"{fc_dir}/sirius_1_6_0/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

#     try:
#         # Read the single region call file
#         single_df = pd.read_table(single_region_file)

#         if not single_df.empty:
#             # Loop through the target gene list and assign the call values to the corresponding columns
#             for gene in Target_PM_gene_list:
#                 gene_df = single_df[single_df['gene'] == gene]
#                 if not gene_df.empty:
#                     df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call_Sirius1_6_0"] = gene_df.iloc[0]['call']

#         else:
#             # Add the index to the list if the dataframe is empty
#             problematic_rows.append(index) 

#     except Exception as e:
#         # Add the index to the list if there is any exception
#         problematic_rows.append(index)  
#         print(f"Error processing file {single_region_file}: {str(e)}")

# # Create a DataFrame with the indices of the problematic rows
# df_problematic = df_VHI01_IST01_01batch_dedup_3.loc[problematic_rows]

# print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is {df_problematic.shape[0]}\n")

# # Show the updated dataframe with the new columns
# df_VHI01_IST01_01batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call_Sirius1_6_0,ARID1A_single_region_call_Sirius1_6_0,ARID1B_single_region_call_Sirius1_6_0,ATM_single_region_call_Sirius1_6_0,ATR_single_region_call_Sirius1_6_0,B2M_single_region_call_Sirius1_6_0,BRCA1_single_region_call_Sirius1_6_0,BRCA2_single_region_call_Sirius1_6_0,BRIP1_single_region_call_Sirius1_6_0,CDH1_single_region_call_Sirius1_6_0,CDK12_single_region_call_Sirius1_6_0,CDKN1B_single_region_call_Sirius1_6_0,CDKN2A_single_region_call_Sirius1_6_0,CHEK1_single_region_call_Sirius1_6_0,CHEK2_single_region_call_Sirius1_6_0,FANCA_single_region_call_Sirius1_6_0,FANCE_single_region_call_Sirius1_6_0,FANCF_single_region_call_Sirius1_6_0,FANCG_single_region_call_Sirius1_6_0,FANCL_single_region_call_Sirius1_6_0,KEAP1_single_region_call_Sirius1_6_0,MGMT_single_region_call_Sirius1_6_0,MLH1_single_region_call_Sirius1_6_0,MSH2_single_region_call_Sirius1_6_0,MSH6_single_region_call_Sirius1_6_0,MTAP_single_region_call_Sirius1_6_0,MUTYH_single_region_call_Sirius1_6_0,NF1_single_region_call_Sirius1_6_0,NF2_single_region_call_Sirius1_6_0,PALB2_single_region_call_Sirius1_6_0,PBRM1_single_region_call_Sirius1_6_0,PMS2_single_region_call_Sirius1_6_0,POLE_single_region_call_Sirius1_6_0,PTEN_single_region_call_Sirius1_6_0,RAD50_single_region_call_Sirius1_6_0,RAD51_single_region_call_Sirius1_6_0,RAD51C_single_region_call_Sirius1_6_0,RAD51D_single_region_call_Sirius1_6_0,RB1_single_region_call_Sirius1_6_0,SDHA_single_region_call_Sirius1_6_0,SDHB_single_region_call_Sirius1_6_0,SDHC_single_region_call_Sirius1_6_0,SDHD_single_region_call_Sirius1_6_0,SMAD4_single_region_call_Sirius1_6_0,STK11_single_region_call_Sirius1_6_0,TP53_single_region_call_Sirius1_6_0,TSC2_single_region_call_Sirius1_6_0
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,Na

In [78]:
# Copying the dataframe
df_VHI01_IST01_01batch_dedup_3 = df_VHI01_IST01_01batch_dedup_2.copy()

# List of target genes
Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
                       "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
                       "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
                       "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
                       "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# Initialize columns for each gene in the dataframe
for gene in Target_PM_gene_list:
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call"] = np.nan
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call_mol"] = np.nan

problematic_rows = []  # List to store indices of problematic rows

# Loop through the rows of the dataframe
for index, crc_row in df_VHI01_IST01_01batch_dedup_3.iterrows():
    fc_dir = crc_row["fc_dir"]
    run_sample_id = crc_row["GHSampleID"]

    # Define the file path for the single region report
    single_region_file = f"{fc_dir}/sirius_1_6_0/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

    try:
        # Read the single region call file
        single_df = pd.read_table(single_region_file)

        if not single_df.empty:
            # Loop through the target gene list and assign the call values to the corresponding columns
            for gene in Target_PM_gene_list:
                gene_df = single_df[single_df['gene'] == gene]
                if not gene_df.empty:
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call"] = gene_df.iloc[0]['call']
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call_mol"] = gene_df.iloc[0]['molecules']
        else:
            # Add the index to the list if the dataframe is empty
            problematic_rows.append(index) 

    except Exception as e:
        # Add the index to the list if there is any exception
        problematic_rows.append(index)  
        print(f"Error processing file {single_region_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_VHI01_IST01_01batch_dedup_3.loc[problematic_rows]

print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is {df_problematic.shape[0]}\n")

# Show the updated dataframe with the new columns
df_VHI01_IST01_01batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call,APC_single_region_call_mol,ARID1A_single_region_call,ARID1A_single_region_call_mol,ARID1B_single_region_call,ARID1B_single_region_call_mol,ATM_single_region_call,ATM_single_region_call_mol,ATR_single_region_call,ATR_single_region_call_mol,B2M_single_region_call,B2M_single_region_call_mol,BRCA1_single_region_call,BRCA1_single_region_call_mol,BRCA2_single_region_call,BRCA2_single_region_call_mol,BRIP1_single_region_call,BRIP1_single_region_call_mol,CDH1_single_region_call,CDH1_single_region_call_mol,CDK12_single_region_call,CDK12_single_region_call_mol,CDKN1B_single_region_call,CDKN1B_single_region_call_mol,CDKN2A_single_region_call,CDKN2A_single_region_call_mol,CHEK1_single_region_call,CHEK1_single_region_call_mol,CHEK2_single_region_call,CHEK2_single_region_call_mol,FANCA_single_region_call,FANCA_single_region_call_mol,FANCE_single_region_call,FANCE_single_region_call_mol,FANCF_single_region_call,FANCF_single_region_call_mol,FANCG_single_region_call,FANCG_single_region_call_mol,FANCL_single_region_call,FANCL_single_region_call_mol,KEAP1_single_region_call,KEAP1_single_region_call_mol,MGMT_single_region_call,MGMT_single_region_call_mol,MLH1_single_region_call,MLH1_single_region_call_mol,MSH2_single_region_call,MSH2_single_region_call_mol,MSH6_single_region_call,MSH6_single_region_call_mol,MTAP_single_region_call,MTAP_single_region_call_mol,MUTYH_single_region_call,MUTYH_single_region_call_mol,NF1_single_region_call,NF1_single_region_call_mol,NF2_single_region_call,NF2_single_region_call_mol,PALB2_single_region_call,PALB2_single_region_call_mol,PBRM1_single_region_call,PBRM1_single_region_call_mol,PMS2_single_region_call,PMS2_single_region_call_mol,POLE_single_region_call,POLE_single_region_call_mol,PTEN_single_region_call,PTEN_single_region_call_mol,RAD50_single_region_call,RAD50_single_region_call_mol,RAD51_single_region_call,RAD51_single_region_call_mol,RAD51C_single_region_call,RAD51C_single_region_call_mol,RAD51D_single_region_call,RAD51D_single_region_call_mol,RB1_single_region_call,RB1_single_region_call_mol,SDHA_single_region_call,SDHA_single_region_call_mol,SDHB_single_region_call,SDHB_single_region_call_mol,SDHC_single_region_call,SDHC_single_region_call_mol,SDHD_single_region_call,SDHD_single_region_call_mol,SMAD4_single_region_call,SMAD4_single_region_call_mol,STK11_single_region_call,STK11_single_region_call_mol,TP53_single_region_call,TP53_single_region_call_mol,TSC2_single_region_call,TSC2_single_region_call_mol
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,

In [80]:
# Retrive the 47 PM gene binary call result

df_VHI01_IST01_01batch_dedup_4 = df_VHI01_IST01_01batch_dedup_3.copy()

# List of 47 LDT reportable PM genes
Target_PM_gene_list = [
    "APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", "CDK12", 
    "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", "FANCL", "KEAP1", 
    "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", "PALB2", "PBRM1", "PMS2", 
    "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", "RB1", "SDHA", "SDHB", "SDHC", 
    "SDHD", "SMAD4", "STK11", "TP53", "TSC2"
]

# Process each gene for each sample
new_rows = []
for index, row in df_VHI01_IST01_01batch_dedup_4.iterrows():
    GHSampleID = row['GHSampleID']
    for gene in Target_PM_gene_list:
        gene_call = row[f'{gene}_single_region_call']
        if pd.notna(gene_call):
            variant_type = 'PromoterMethylationDetected' if gene_call == 1 else 'PromoterMethylationNotDetected'
            new_rows.append({'GHSampleID': GHSampleID, 'Variant_type': variant_type, 'Gene': gene})

# Append new rows to the dataframe
df_47_PM_gene = pd.DataFrame(new_rows)
df_47_PM_gene.head()


,GHSampleID,Variant_type,Gene
0,B00587013,PromoterMethylationNotDetected,APC
1,B00587013,PromoterMethylationNotDetected,ARID1A
2,B00587013,PromoterMethylationNotDetected,ARID1B
3,B00587013,PromoterMethylationNotDetected,ATM
4,B00587013,PromoterMethylationNotDetected,ATR


In [ ]:
# Check any edge case for reportable PM when tumor methylation status is dectected
# Promoter_methylation_count > 5 & Alt_NoAlt == 1 & Max_percentage > 1%
# This is the confluence page: https://guardanthealth.atlassian.net/wiki/spaces/PHARMA/pages/3027927678/Part+II.+Automated+review+by+the+pipeline 

df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3[(df_VHI01_IST01_01batch_dedup_3['Tumor_methylation_status'] == 'Not Detected') & (df_VHI01_IST01_01batch_dedup_3['Alt-NoAlt'] == 1) & (df_VHI01_IST01_01batch_dedup_3['Max_percentage'] > 1)]
# Select only columns that match the pattern '{gene}_single_region_call_mol'
numeric_columns = [col for col in df_VHI01_IST01_01batch_dedup_3_edge.columns if col.endswith("_single_region_call_mol")]

# Apply filter: Keep rows where any of these numeric columns have a value greater than 5
df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3_edge[(df_VHI01_IST01_01batch_dedup_3_edge[numeric_columns] > 5).any(axis=1)]
print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with potential reportable PM edge cases is {df_VHI01_IST01_01batch_dedup_3_edge.shape[0]}\n")

# Print out the edge PM cases
filtered_results = []

# Iterate over each row that meets the condition
for index, row in df_VHI01_IST01_01batch_dedup_3_edge.iterrows():
    GHSampleID = row["GHSampleID"]  # Extract the sample ID
    for col in numeric_columns:
        if row[col] > 5:  # Check if value is greater than 5
            # Derive the corresponding "_single_region_call" column name
            gene_name = col.replace("_single_region_call_mol", "")
            single_region_call_col = f"{gene_name}_single_region_call"

            # Check if this column exists in the DataFrame before retrieving its value
            single_region_call_value = row[single_region_call_col] if single_region_call_col in row else "N/A"

            # Append to results
            filtered_results.append({
                "GHSampleID": GHSampleID,
                "Column": col,
                "Value": row[col],
                "Single_Region_Call_Column": single_region_call_col,
                "Single_Region_Call_Value": single_region_call_value
            })

# Convert results to DataFrame for easy viewing
df_filtered_results = pd.DataFrame(filtered_results)
df_filtered_results

Total sample size of VHI01_IST01_Infinity Infinity batch with reportable PM edge cases is 0



""


In [ ]:
# # Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# # Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
# df_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] == 'PromoterMethylation'].copy()

# # Step 2: Remove these rows from the original dataframe to avoid duplication later
# df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()

# # Step 3: Include the 'Study_ID', 'Variant_type', 'Gene' columns and other necessary columns
# df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
#                                                            'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
#                                                            'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
#                                                            'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
#                                                            'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
#                                                            'Variant_type', 'Gene']].copy()

# # Step 4: Merge with df_47_PM_gene using a left join, keeping GHSampleID match and suffixes to avoid conflicts
# df_promoter_methylation_updated = df_promoter_methylation_updated.merge(df_47_PM_gene, on='GHSampleID', how='left', suffixes=('', '_PM'))

# # Step 5: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
# df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
# df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# # Step 6: Drop the temporary columns generated during the merge
# df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# # Step 7: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
# ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
# all_ghsampleid = df_VHI01_IST01_01batch['GHSampleID'].unique()
# missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# # Step 8: For these missing GHSampleIDs, manually create the "PromoterMethylation" rows using df_47_PM_gene
# df_missing_pm_rows = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(missing_ghsampleid)].copy()

# # Add the necessary columns with NaN or default values for the missing GHSampleID samples
# for col in ['Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
#             'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
#             'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
#             'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
#             'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name']:
#     df_missing_pm_rows[col] = None  # Assign default values or NaN for these columns

# # Step 9: Append the missing rows to the updated promoter methylation rows
# df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# # Step 10: Append the updated rows back to the original dataframe while keeping their positions
# df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# # Step 11: Sort the dataframe by GHSampleID and place rows accordingly
# df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# # Step 12: Ensure there are no duplicates
# df_ori_with_47pm.drop_duplicates(inplace=True)

# df_ori_with_47pm.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
655,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
656,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,TET2,4,106156445.0,3,V449A,T>C,c.1346T>C,NM_001127208.3,52.75,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs753698541,NaN,NaN,NaN,NaN,6245.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
657,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FAAP100,17,79511105.0,7,A784A,G>A,c.2352C>T,NM_025161.6,51.45,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs142183217,NaN,NaN,NaN,NaN,5688.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
658,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,LRP5,11,68177472.0,10,M728L,A>T,c.2182A>T,NM_002335.4,51.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs149241739,NaN,NaN,NaN,NaN,5526.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
659,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FOXA1,14,38060693.0,2,T432T,C>G,c.1296G>C,NM_004496.5,51.06,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375938424,NaN,NaN,NaN,NaN,5745.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable


In [83]:
# Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
df_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] == 'PromoterMethylation'].copy()

# Step 2: Remove these rows from the original dataframe to avoid duplication later
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()

# Step 3: Include the necessary columns
df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
                                                           'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
                                                           'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
                                                           'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
                                                           'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
                                                           'Variant_type', 'Gene']].copy()

# Step 4: Filter out invalid GHSampleIDs based on Sample_status and Tumor_methylation_status
# Based on SOP, Not Detected case are more likely to suppress all the PM calls
valid_ghsampleid = df_VHI01_IST01_01batch[
    (df_VHI01_IST01_01batch['Sample_status'] != 'FAIL') & 
    (df_VHI01_IST01_01batch['Tumor_methylation_status'] != 'Not Detected')
]['GHSampleID'].unique()

# Retain only rows with valid GHSampleIDs in df_promoter_methylation_updated
df_promoter_methylation_updated = df_promoter_methylation_updated[
    df_promoter_methylation_updated['GHSampleID'].isin(valid_ghsampleid)
]

# Step 5: Filter df_47_PM_gene to only include valid GHSampleIDs
df_47_PM_gene_filtered = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(valid_ghsampleid)].copy()

# Step 6: Merge with the filtered df_47_PM_gene using a left join
df_promoter_methylation_updated = df_promoter_methylation_updated.merge(
    df_47_PM_gene_filtered, on='GHSampleID', how='left', suffixes=('', '_PM')
)

# Step 7: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# Step 8: Drop the temporary columns generated during the merge
df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# Step 9: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
all_ghsampleid = df_VHI01_IST01_01batch['GHSampleID'].unique()
missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# Filter missing GHSampleIDs to only include valid ones
missing_ghsampleid = missing_ghsampleid.intersection(valid_ghsampleid)

# Step 10: Create the "PromoterMethylation" rows for the filtered GHSampleIDs
df_missing_pm_rows = df_47_PM_gene_filtered[df_47_PM_gene_filtered['GHSampleID'].isin(missing_ghsampleid)].copy()

# Populate the necessary columns by retrieving values for the same GHSampleID
columns_to_populate = [
    'Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
    'Sample_status', 'Sample_comment', 'Max_percentage', 'Tumor_methylation_status', 'Tumor_methylation_score', 
    'TMB_score', 'TMB_category', 'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 
    'Received_date', 'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name'
]

for col in columns_to_populate:
    if col in df_VHI01_IST01_01batch.columns:
        df_missing_pm_rows[col] = df_missing_pm_rows['GHSampleID'].map(
            df_VHI01_IST01_01batch.groupby('GHSampleID')[col].first()
        )
    else:
        print(f"Column {col} not found in source dataframe, leaving as NaN")

# Step 11: Append the missing rows to the updated promoter methylation rows
df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# Step 12: Append the updated rows back to the original dataframe while keeping their positions
df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# Step 13: Sort the dataframe by GHSampleID and place rows accordingly
df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# Step 14: Ensure there are no duplicates
df_ori_with_47pm.drop_duplicates(inplace=True)

# Display the updated dataframe
df_ori_with_47pm.head()


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
655,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
656,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,TET2,4,106156445.0,3,V449A,T>C,c.1346T>C,NM_001127208.3,52.75,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs753698541,NaN,NaN,NaN,NaN,6245.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
657,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FAAP100,17,79511105.0,7,A784A,G>A,c.2352C>T,NM_025161.6,51.45,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs142183217,NaN,NaN,NaN,NaN,5688.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
658,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,LRP5,11,68177472.0,10,M728L,A>T,c.2182A>T,NM_002335.4,51.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs149241739,NaN,NaN,NaN,NaN,5526.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
659,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FOXA1,14,38060693.0,2,T432T,C>G,c.1296G>C,NM_004496.5,51.06,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375938424,NaN,NaN,NaN,NaN,5745.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable


In [84]:
df_ori_with_47pm_B00586988 = df_ori_with_47pm[df_ori_with_47pm['GHSampleID'] == 'B00586988']
df_ori_with_47pm_B00586988

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
74,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.5,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable


In [ ]:
# Check the result 
print(df_VHI01_IST01_01batch.shape)
print(df_47_PM_gene.shape)
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()
print(df_non_promoter_methylation.shape)
print(df_ori_with_47pm.shape)

# Save the temporary data
df_ori_with_47pm.to_csv(f"{work_dir}/df_ori_with_47pm_batch_all.csv", index = False)

(11815, 56)
(6909, 3)
(9884, 56)
(16464, 56)


In [ ]:
# Additional dataframe updates based on Rachael's request
# Drop two columns
df_ori_with_47pm_removed_2col = df_ori_with_47pm.drop(columns=['HRD_score', 'Alleletype'])

# Drop IO Allele type info
df_final = df_ori_with_47pm_removed_2col[df_ori_with_47pm_removed_2col['Variant_type'] != "AlleleType"]

# Save the data
df_final.to_csv(f"{work_dir}/df_final_batch_all.csv", index = False)
df_final.shape

(16464, 54)

# Add 47 gene promoter methylation in to the report VHI01_IST01_Infinity_03 (02/07/2025)

## Data preocessing

In [13]:
# Readin data

df_VHI01_IST01_01batch = pd.read_csv(f"{work_dir}/20250207_GuardantHealthIST_IST01_Batch3_118Samples_InfinityReport.csv", header = 1) 

print("Total sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.shape[0]) + "\n") 
print("Total unique sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.Patient_ID.nunique()) + "\n")

df_VHI01_IST01_01batch.head()

Total sample size of df_VHI01_IST01_01batch Infinity batch is 7483

Total unique sample size of df_VHI01_IST01_01batch Infinity batch is 118

Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is 83



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
1,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,PRDM1,6,106553592.0,5,A519A,G>A,c.1557G>A,NM_001198.4,51.60,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV64845075,rs138916112,NaN,NaN,NaN,NaN,5344.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
2,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,FLT4,5,180048651.0,13,S637R,G>C,c.1911C>G,NM_002020.5,50.80,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV56106934,rs148898412,NaN,NaN,NaN,NaN,5002.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
3,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,APC,5,112176485.0,16,M1732L,A>T,c.5194A>T,NM_000038.6,50.20,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain_significance,617174.0,NaN,NaN,4935.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
4,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,NF2,22,30000061.0,1,R25K,G>A,c.74G>A,NM_000268.4,50.06,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain_significance,574061.0,NaN,NaN,3345.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable


In [14]:
# Remove duplication
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch.drop_duplicates(subset = "GHSampleID")
df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
46,mCRPC_BIOBANK,GU3111,A1273172,B00909952,PRO207,Progression,1,SUCCESS,NaN,SNV,NaN,TYMP,22,50964895.0,7,L313L,C>T,c.939G>A,NM_001113756.3,55.29,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1414.0,0.51,NaN,0.019688,Detected,0.004177,NaN,Not evaluable,Not Detected,29.20,2.5,0.0,01/16/2025,06/13/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
90,mCRPC_BIOBANK,GU2264,A1273173,B00912113,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,MERTK,2,112705105.0,4,E240K,G>A,c.718G>A,NM_006343.3,52.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV54927603,rs868703580,NaN,NaN,NaN,NaN,4370.0,8.19,NaN,0.022271,Detected,0.717642,NaN,Not evaluable,Not Detected,49.13,1.5,0.0,01/16/2025,06/09/2022,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
178,mCRPC_BIOBANK,GU1055,A1273174,B00911090,PRO016,Baseline,1,SUCCESS,NaN,SNV,NaN,MAP3K13,3,185198208.0,13,I897T,T>C,c.2690T>C,NM_004721.5,51.47,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53987036,rs372763568,NaN,NaN,NaN,NaN,4964.0,4.87,NaN,0.009195,Detected,0.088850,7.59,Low,Not Detected,24.82,2.5,0.0,01/16/2025,07/10/2020,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
232,mCRPC_BIOBANK,GU3752,A1273175,B00912118,PRO167,Progression,1,SUCCESS,NaN,SNV,NaN,REV3L,6,111696540.0,14,L1006L,T>C,c.3018A>G,NM_002912.5,52.51,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs767320584,NaN,NaN,NaN,NaN,2370.0,10.35,NaN,0.022506,Detected,0.010723,7.59,Low,Not Detected,20.59,3.0,0.0,01/16/2025,03/04/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable


In [15]:
# Merge runid based on re-run BART v0 https://docs.google.com/spreadsheets/d/1knCW-Jnu1lAiOTkUYTZrFAgyIMR5onjZDwJ5U-mtiNE/edit?gid=0#gid=0
# Readin the file with runid
df_VHI01_IST01_01batch_runid = pd.read_csv(f"{work_dir}/VHI01_IST01_Infinity_03_v0.csv", header = 3) 
df_VHI01_IST01_01batch_runid = df_VHI01_IST01_01batch_runid.drop_duplicates(subset = 'run_sample_id')
df_VHI01_IST01_01batch_runid.head()

# Merge the runid back
df_VHI01_IST01_01batch_runid.rename(columns={'run_sample_id': 'GHSampleID'}, inplace=True)
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch_dedup.merge(df_VHI01_IST01_01batch_runid[["GHSampleID", "runid"]], on = "GHSampleID", how = "left")

df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF
1,mCRPC_BIOBANK,GU3111,A1273172,B00909952,PRO207,Progression,1,SUCCESS,NaN,SNV,NaN,TYMP,22,50964895.0,7,L313L,C>T,c.939G>A,NM_001113756.3,55.29,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1414.0,0.51,NaN,0.019688,Detected,0.004177,NaN,Not evaluable,Not Detected,29.20,2.5,0.0,01/16/2025,06/13/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250125_A00277_1224_BH25KGDSXF
2,mCRPC_BIOBANK,GU2264,A1273173,B00912113,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,MERTK,2,112705105.0,4,E240K,G>A,c.718G>A,NM_006343.3,52.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV54927603,rs868703580,NaN,NaN,NaN,NaN,4370.0,8.19,NaN,0.022271,Detected,0.717642,NaN,Not evaluable,Not Detected,49.13,1.5,0.0,01/16/2025,06/09/2022,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF
3,mCRPC_BIOBANK,GU1055,A1273174,B00911090,PRO016,Baseline,1,SUCCESS,NaN,SNV,NaN,MAP3K13,3,185198208.0,13,I897T,T>C,c.2690T>C,NM_004721.5,51.47,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53987036,rs372763568,NaN,NaN,NaN,NaN,4964.0,4.87,NaN,0.009195,Detected,0.088850,7.59,Low,Not Detected,24.82,2.5,0.0,01/16/2025,07/10/2020,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF
4,mCRPC_BIOBANK,GU3752,A1273175,B00912118,PRO167,Progression,1,SUCCESS,NaN,SNV,NaN,REV3L,6,111696540.0,14,L1006L,T>C,c.3018A>G,NM_002912.5,52.51,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs767320584,NaN,NaN,NaN,NaN,2370.0,10.35,NaN,0.022506,Detected,0.010723,7.59,Low,Not Detected,20.59,3.0,0.0,01/16/2025,03/04/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF


In [16]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_VHI01_IST01_01batch_dedup_2, df_no_found = add_fc_dir_column(df_VHI01_IST01_01batch_dedup)

print("Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is " + str(df_VHI01_IST01_01batch_dedup_2.shape[0]) + "\n") 
print("Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_VHI01_IST01_01batch_dedup_2.head()

Total sample size of VHI01_IST01_Infinity Infinity batch with fc_dir is 118

Total sample size of VHI01_IST01_Infinity Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF,/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius
1,mCRPC_BIOBANK,GU3111,A1273172,B00909952,PRO207,Progression,1,SUCCESS,NaN,SNV,NaN,TYMP,22,50964895.0,7,L313L,C>T,c.939G>A,NM_001113756.3,55.29,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1414.0,0.51,NaN,0.019688,Detected,0.004177,NaN,Not evaluable,Not Detected,29.20,2.5,0.0,01/16/2025,06/13/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250125_A00277_1224_BH25KGDSXF,/ghsfa/ivd/flowcentral/250125_A00277_1224_BH25KGDSXF.cbb750c2-c173-4d59-9da5-d4e4b3be566f.20250127142438/sirius
2,mCRPC_BIOBANK,GU2264,A1273173,B00912113,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,MERTK,2,112705105.0,4,E240K,G>A,c.718G>A,NM_006343.3,52.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV54927603,rs868703580,NaN,NaN,NaN,NaN,4370.0,8.19,NaN,0.022271,Detected,0.717642,NaN,Not evaluable,Not Detected,49.13,1.5,0.0,01/16/2025,06/09/2022,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF,/ghsfa/ivd/flowcentral/250126_A00770_1124_BH25LJDSXF.02cabc46-d2d7-4199-b668-1794759155c3.20250128114740/sirius
3,mCRPC_BIOBANK,GU1055,A1273174,B00911090,PRO016,Baseline,1,SUCCESS,NaN,SNV,NaN,MAP3K13,3,185198208.0,13,I897T,T>C,c.2690T>C,NM_004721.5,51.47,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53987036,rs372763568,NaN,NaN,NaN,NaN,4964.0,4.87,NaN,0.009195,Detected,0.088850,7.59,Low,Not Detected,24.82,2.5,0.0,01/16/2025,07/10/2020,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF,/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius
4,mCRPC_BIOBANK,GU3752,A1273175,B00912118,PRO167,Progression,1,SUCCESS,NaN,SNV,NaN,REV3L,6,111696540.0,14,L1006L,T>C,c.3018A>G,NM_002912.5,52.51,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs767320584,NaN,NaN,NaN,NaN,2370.0,10.35,NaN,0.022506,Detected,0.010723,7.59,Low,Not Detected,20.59,3.0,0.0,01/16/2025,03/04/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF,/ghsfa/ivd/flowcentral/250126_A00770_1124_BH25LJDSXF.02cabc46-d2d7-4199-b668-1794759155c3.20250128114740/sirius


In [17]:
# Save the data for cummulative intermediate file
df_VHI01_IST01_01batch_dedup_2.to_csv(f"{work_dir}/df_VHI01_IST01_01batch_dedup_2_batch3_118_samples_with_fc_dir_02202025.csv", index = False)

## Generate Pancancer S3 MB caller and Single region caller outputs for this SOW
#### Notes:
    1. No cancer type provided
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [16]:
# Copying the dataframe
df_VHI01_IST01_01batch_dedup_3 = df_VHI01_IST01_01batch_dedup_2.copy()

# List of target genes
Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
                       "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
                       "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
                       "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
                       "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# Initialize columns for each gene in the dataframe
for gene in Target_PM_gene_list:
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call"] = np.nan
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call_mol"] = np.nan

problematic_rows = []  # List to store indices of problematic rows

# Loop through the rows of the dataframe
for index, crc_row in df_VHI01_IST01_01batch_dedup_3.iterrows():
    fc_dir = crc_row["fc_dir"]
    run_sample_id = crc_row["GHSampleID"]

    # Define the file path for the single region report
    single_region_file = f"{fc_dir}/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

    try:
        # Read the single region call file
        single_df = pd.read_table(single_region_file)

        if not single_df.empty:
            # Loop through the target gene list and assign the call values to the corresponding columns
            for gene in Target_PM_gene_list:
                gene_df = single_df[single_df['gene'] == gene]
                if not gene_df.empty:
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call"] = gene_df.iloc[0]['call']
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call_mol"] = gene_df.iloc[0]['molecules']
        else:
            # Add the index to the list if the dataframe is empty
            problematic_rows.append(index) 

    except Exception as e:
        # Add the index to the list if there is any exception
        problematic_rows.append(index)  
        print(f"Error processing file {single_region_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_VHI01_IST01_01batch_dedup_3.loc[problematic_rows]

print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is {df_problematic.shape[0]}\n")

# Show the updated dataframe with the new columns
df_VHI01_IST01_01batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call,APC_single_region_call_mol,ARID1A_single_region_call,ARID1A_single_region_call_mol,ARID1B_single_region_call,ARID1B_single_region_call_mol,ATM_single_region_call,ATM_single_region_call_mol,ATR_single_region_call,ATR_single_region_call_mol,B2M_single_region_call,B2M_single_region_call_mol,BRCA1_single_region_call,BRCA1_single_region_call_mol,BRCA2_single_region_call,BRCA2_single_region_call_mol,BRIP1_single_region_call,BRIP1_single_region_call_mol,CDH1_single_region_call,CDH1_single_region_call_mol,CDK12_single_region_call,CDK12_single_region_call_mol,CDKN1B_single_region_call,CDKN1B_single_region_call_mol,CDKN2A_single_region_call,CDKN2A_single_region_call_mol,CHEK1_single_region_call,CHEK1_single_region_call_mol,CHEK2_single_region_call,CHEK2_single_region_call_mol,FANCA_single_region_call,FANCA_single_region_call_mol,FANCE_single_region_call,FANCE_single_region_call_mol,FANCF_single_region_call,FANCF_single_region_call_mol,FANCG_single_region_call,FANCG_single_region_call_mol,FANCL_single_region_call,FANCL_single_region_call_mol,KEAP1_single_region_call,KEAP1_single_region_call_mol,MGMT_single_region_call,MGMT_single_region_call_mol,MLH1_single_region_call,MLH1_single_region_call_mol,MSH2_single_region_call,MSH2_single_region_call_mol,MSH6_single_region_call,MSH6_single_region_call_mol,MTAP_single_region_call,MTAP_single_region_call_mol,MUTYH_single_region_call,MUTYH_single_region_call_mol,NF1_single_region_call,NF1_single_region_call_mol,NF2_single_region_call,NF2_single_region_call_mol,PALB2_single_region_call,PALB2_single_region_call_mol,PBRM1_single_region_call,PBRM1_single_region_call_mol,PMS2_single_region_call,PMS2_single_region_call_mol,POLE_single_region_call,POLE_single_region_call_mol,PTEN_single_region_call,PTEN_single_region_call_mol,RAD50_single_region_call,RAD50_single_region_call_mol,RAD51_single_region_call,RAD51_single_region_call_mol,RAD51C_single_region_call,RAD51C_single_region_call_mol,RAD51D_single_region_call,RAD51D_single_region_call_mol,RB1_single_region_call,RB1_single_region_call_mol,SDHA_single_region_call,SDHA_single_region_call_mol,SDHB_single_region_call,SDHB_single_region_call_mol,SDHC_single_region_call,SDHC_single_region_call_mol,SDHD_single_region_call,SDHD_single_region_call_mol,SMAD4_single_region_call,SMAD4_single_region_call_mol,STK11_single_region_call,STK11_single_region_call_mol,TP53_single_region_call,TP53_single_region_call_mol,TSC2_single_region_call,TSC2_single_region_call_mol
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF,/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
# Check any edge case for reportable PM when tumor methylation status is dectected
# Promoter_methylation_count > 5 & Alt_NoAlt == 1 & Max_percentage > 1%
# This is the confluence page: https://guardanthealth.atlassian.net/wiki/spaces/PHARMA/pages/3027927678/Part+II.+Automated+review+by+the+pipeline 

df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3[(df_VHI01_IST01_01batch_dedup_3['Tumor_methylation_status'] == 'Not Detected') & (df_VHI01_IST01_01batch_dedup_3['Alt-NoAlt'] == 1) & (df_VHI01_IST01_01batch_dedup_3['Max_percentage'] > 1)]
# Select only columns that match the pattern '{gene}_single_region_call_mol'
numeric_columns = [col for col in df_VHI01_IST01_01batch_dedup_3_edge.columns if col.endswith("_single_region_call_mol")]

# Apply filter: Keep rows where any of these numeric columns have a value greater than 5
df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3_edge[(df_VHI01_IST01_01batch_dedup_3_edge[numeric_columns] > 5).any(axis=1)]
print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with potential reportable PM edge cases is {df_VHI01_IST01_01batch_dedup_3_edge.shape[0]}\n")

# Print out the edge PM cases
filtered_results = []

# Iterate over each row that meets the condition
for index, row in df_VHI01_IST01_01batch_dedup_3_edge.iterrows():
    GHSampleID = row["GHSampleID"]  # Extract the sample ID
    for col in numeric_columns:
        if row[col] > 5:  # Check if value is greater than 5
            # Derive the corresponding "_single_region_call" column name
            gene_name = col.replace("_single_region_call_mol", "")
            single_region_call_col = f"{gene_name}_single_region_call"

            # Check if this column exists in the DataFrame before retrieving its value
            single_region_call_value = row[single_region_call_col] if single_region_call_col in row else "N/A"

            # Append to results
            filtered_results.append({
                "GHSampleID": GHSampleID,
                "Column": col,
                "Value": row[col],
                "Single_Region_Call_Column": single_region_call_col,
                "Single_Region_Call_Value": single_region_call_value
            })

# Convert results to DataFrame for easy viewing
df_filtered_results = pd.DataFrame(filtered_results)
df_filtered_results

Total sample size of VHI01_IST01_Infinity Infinity batch with reportable PM edge cases is 4



,GHSampleID,Column,Value,Single_Region_Call_Column,Single_Region_Call_Value
0,B00911088,CDH1_single_region_call_mol,11.0,CDH1_single_region_call,0.0
1,B00912115,BRIP1_single_region_call_mol,10.0,BRIP1_single_region_call,0.0
2,B00911114,CDH1_single_region_call_mol,6.0,CDH1_single_region_call,0.0
3,B00910016,CDH1_single_region_call_mol,7.0,CDH1_single_region_call,0.0
4,B00910016,PMS2_single_region_call_mol,7.0,PMS2_single_region_call,0.0
5,B00910016,RAD51_single_region_call_mol,6.0,RAD51_single_region_call,0.0


In [ ]:
# Based on above cell result, none of the potential reportable PM in methylation status not detected case are realy reportbale!

In [24]:
# List to store filtered results
filtered_results = []

# Iterate over each row that meets the condition
for index, row in df_VHI01_IST01_01batch_dedup_3_edge.iterrows():
    GHSampleID = row["GHSampleID"]  # Extract the sample ID
    for col in numeric_columns:
        if row[col] > 5:  # Check if value is greater than 5
            # Derive the corresponding "_single_region_call" column name
            gene_name = col.replace("_single_region_call_mol", "")
            single_region_call_col = f"{gene_name}_single_region_call"

            # Check if this column exists in the DataFrame before retrieving its value
            single_region_call_value = row[single_region_call_col] if single_region_call_col in row else "N/A"

            # Append to results
            filtered_results.append({
                "GHSampleID": GHSampleID,
                "Column": col,
                "Value": row[col],
                "Single_Region_Call_Column": single_region_call_col,
                "Single_Region_Call_Value": single_region_call_value
            })

# Convert results to DataFrame for easy viewing
df_filtered_results = pd.DataFrame(filtered_results)

df_filtered_results

,GHSampleID,Column,Value,Single_Region_Call_Column,Single_Region_Call_Value
0,B00911088,CDH1_single_region_call_mol,11.0,CDH1_single_region_call,0.0
1,B00912115,BRIP1_single_region_call_mol,10.0,BRIP1_single_region_call,0.0
2,B00911114,CDH1_single_region_call_mol,6.0,CDH1_single_region_call,0.0
3,B00910016,CDH1_single_region_call_mol,7.0,CDH1_single_region_call,0.0
4,B00910016,PMS2_single_region_call_mol,7.0,PMS2_single_region_call,0.0
5,B00910016,RAD51_single_region_call_mol,6.0,RAD51_single_region_call,0.0


In [26]:
# Retrive the 47 PM gene binary call result

df_VHI01_IST01_01batch_dedup_4 = df_VHI01_IST01_01batch_dedup_3.copy()

# List of 47 LDT reportable PM genes
Target_PM_gene_list = [
    "APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", "CDK12", 
    "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", "FANCL", "KEAP1", 
    "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", "PALB2", "PBRM1", "PMS2", 
    "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", "RB1", "SDHA", "SDHB", "SDHC", 
    "SDHD", "SMAD4", "STK11", "TP53", "TSC2"
]

# Process each gene for each sample
new_rows = []
for index, row in df_VHI01_IST01_01batch_dedup_4.iterrows():
    GHSampleID = row['GHSampleID']
    for gene in Target_PM_gene_list:
        gene_call = row[f'{gene}_single_region_call']
        if pd.notna(gene_call):
            variant_type = 'PromoterMethylationDetected' if gene_call == 1 else 'PromoterMethylationNotDetected'
            new_rows.append({'GHSampleID': GHSampleID, 'Variant_type': variant_type, 'Gene': gene})

# Append new rows to the dataframe
df_47_PM_gene = pd.DataFrame(new_rows)
print(df_47_PM_gene.shape)
df_47_PM_gene.head()


(5546, 3)


,GHSampleID,Variant_type,Gene
0,B00911075,PromoterMethylationNotDetected,APC
1,B00911075,PromoterMethylationNotDetected,ARID1A
2,B00911075,PromoterMethylationNotDetected,ARID1B
3,B00911075,PromoterMethylationNotDetected,ATM
4,B00911075,PromoterMethylationNotDetected,ATR


In [19]:
# Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
df_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] == 'PromoterMethylation'].copy()

# Step 2: Remove these rows from the original dataframe to avoid duplication later
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()

# Step 3: Include the necessary columns
df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
                                                           'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
                                                           'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
                                                           'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
                                                           'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
                                                           'Variant_type', 'Gene']].copy()

# Step 4: Filter out invalid GHSampleIDs based on Sample_status and Tumor_methylation_status
# Based on SOP, Not Detected case are more likely to suppress all the PM calls
valid_ghsampleid = df_VHI01_IST01_01batch[
    (df_VHI01_IST01_01batch['Sample_status'] != 'FAIL') & 
    (df_VHI01_IST01_01batch['Tumor_methylation_status'] != 'Not Detected')
]['GHSampleID'].unique()

# Retain only rows with valid GHSampleIDs in df_promoter_methylation_updated
df_promoter_methylation_updated = df_promoter_methylation_updated[
    df_promoter_methylation_updated['GHSampleID'].isin(valid_ghsampleid)
]

# Step 5: Filter df_47_PM_gene to only include valid GHSampleIDs
df_47_PM_gene_filtered = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(valid_ghsampleid)].copy()

# Step 6: Merge with the filtered df_47_PM_gene using a left join
df_promoter_methylation_updated = df_promoter_methylation_updated.merge(
    df_47_PM_gene_filtered, on='GHSampleID', how='left', suffixes=('', '_PM')
)

# Step 7: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# Step 8: Drop the temporary columns generated during the merge
df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# Step 9: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
all_ghsampleid = df_VHI01_IST01_01batch['GHSampleID'].unique()
missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# Filter missing GHSampleIDs to only include valid ones
missing_ghsampleid = missing_ghsampleid.intersection(valid_ghsampleid)

# Step 10: Create the "PromoterMethylation" rows for the filtered GHSampleIDs
df_missing_pm_rows = df_47_PM_gene_filtered[df_47_PM_gene_filtered['GHSampleID'].isin(missing_ghsampleid)].copy()

# Populate the necessary columns by retrieving values for the same GHSampleID
columns_to_populate = [
    'Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
    'Sample_status', 'Sample_comment', 'Max_percentage', 'Tumor_methylation_status', 'Tumor_methylation_score', 
    'TMB_score', 'TMB_category', 'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 
    'Received_date', 'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name'
]

for col in columns_to_populate:
    if col in df_VHI01_IST01_01batch.columns:
        df_missing_pm_rows[col] = df_missing_pm_rows['GHSampleID'].map(
            df_VHI01_IST01_01batch.groupby('GHSampleID')[col].first()
        )
    else:
        print(f"Column {col} not found in source dataframe, leaving as NaN")

# Step 11: Append the missing rows to the updated promoter methylation rows
df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# Step 12: Append the updated rows back to the original dataframe while keeping their positions
df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# Step 13: Sort the dataframe by GHSampleID and place rows accordingly
df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# Step 14: Ensure there are no duplicates
df_ori_with_47pm.drop_duplicates(inplace=True)

# Display the updated dataframe
df_ori_with_47pm.head()


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
718,mCRPC_BIOBANK,GU2827,A1273184,B00909932,PRO165,Progression,1,SUCCESS,NaN,SNV,NaN,CTC1,17,8151331.0,1,V8V,G>T,c.24C>A,NM_025099.6,54.86,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs749818659,NaN,NaN,NaN,NaN,551.0,0.83,NaN,0.021246,Detected,0.000566,7.12,Low,Not Detected,12.04,2.5,0.0,01/16/2025,02/14/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
719,mCRPC_BIOBANK,GU2827,A1273184,B00909932,PRO165,Progression,1,SUCCESS,NaN,SNV,NaN,STK11,19,1221319.0,6,P281L,C>T,c.842C>T,NM_000455.5,52.24,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV58821563,rs121913322,Conflicting_interpretations_of_pathogenicity,151829.0,NaN,NaN,1497.0,0.83,NaN,0.021246,Detected,0.000566,7.12,Low,Not Detected,12.04,2.5,0.0,01/16/2025,02/14/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
720,mCRPC_BIOBANK,GU2827,A1273184,B00909932,PRO165,Progression,1,SUCCESS,NaN,SNV,NaN,APC,5,112177801.0,16,P2170P,A>C,c.6510A>C,NM_000038.6,50.97,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs138571760,Benign/Likely_benign,182447.0,NaN,NaN,1367.0,0.83,NaN,0.021246,Detected,0.000566,7.12,Low,Not Detected,12.04,2.5,0.0,01/16/2025,02/14/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
721,mCRPC_BIOBANK,GU2827,A1273184,B00909932,PRO165,Progression,1,SUCCESS,NaN,SNV,NaN,RIF1,2,152319637.0,29,S1201S,A>C,c.3603A>C,NM_001177665.2,50.46,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs770430488,NaN,NaN,NaN,NaN,1201.0,0.83,NaN,0.021246,Detected,0.000566,7.12,Low,Not Detected,12.04,2.5,0.0,01/16/2025,02/14/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable
722,mCRPC_BIOBANK,GU2827,A1273184,B00909932,PRO165,Progression,1,SUCCESS,NaN,SNV,NaN,FCGR2A,1,161476204.0,3,Q63W,CA>TG,c.187_188delCAinsTG,NM_001136219.3,49.79,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs201218628,NaN,NaN,NaN,NaN,1407.0,0.83,NaN,0.021246,Detected,0.000566,7.12,Low,Not Detected,12.04,2.5,0.0,01/16/2025,02/14/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable


In [20]:
# Check failed cases
df_ori_with_47pm_fail = df_ori_with_47pm[df_ori_with_47pm['GHSampleID'] == 'B00911129']
df_ori_with_47pm_fail

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
1373,mCRPC_BIOBANK,GU4214/ESR,A1273198,B00911129,PRO359,Baseline,0,FAIL,"Methylation Fail, Low Diversity",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.54,2.0,0.0,01/16/2025,08/12/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable


In [21]:
# Check the result 
print(df_VHI01_IST01_01batch.shape)
print(df_47_PM_gene.shape)
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()
print(df_non_promoter_methylation.shape)
print(df_ori_with_47pm.shape)

# Save the temporary data
df_ori_with_47pm.to_csv(f"{work_dir}/df_ori_with_47pm_batch_all_02072025.csv", index = False)

(7483, 56)
(5546, 3)
(6344, 56)
(11279, 56)


In [22]:
# Additional dataframe updates based on Rachael's request
# Drop two columns
df_ori_with_47pm_removed_2col = df_ori_with_47pm.drop(columns=['HRD_score', 'Alleletype'])

# Drop IO Allele type info
df_final = df_ori_with_47pm_removed_2col[df_ori_with_47pm_removed_2col['Variant_type'] != "AlleleType"]

# Save the data
df_final.to_csv(f"{work_dir}/df_final_batch_all_02072025.csv", index = False)
df_final.shape

(11279, 54)

# Add 47 gene promoter methylation in to the report all 3 batches VHI01_IST01_Infinity_batch1to3 (02/19/2025 after correcting all the inconsistent germline/somatic or somatic/somatic putative CH)

## Data preocessing

In [47]:
# Readin data

df_VHI01_IST01_01batch = pd.read_csv(f"/home/hazhang/projects/Reporting/email_summary_reports/20250219_GuardantHealthIST_IST01_Batch1to3_265Samples_InfinityReport_new.csv", header = 1) 

print("Total sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.shape[0]) + "\n") 
print("Total unique sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is " + str(df_VHI01_IST01_01batch.Patient_ID.nunique()) + "\n")

df_VHI01_IST01_01batch.head()

Total sample size of df_VHI01_IST01_01batch Infinity batch is 19298

Total unique sample size of df_VHI01_IST01_01batch Infinity batch is 265

Total unique patient sample size of df_VHI01_IST01_01batch Infinity batch is 147



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
1,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187539925.0,10,I2605I,G>A,c.7815C>T,NM_005245.4,52.40,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71673524,rs750802295,NaN,NaN,NaN,NaN,4970.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
2,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,PIM1,6,37138190.0,1,S38T,T>A,c.112T>A,NM_001243186.1,51.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1042528446,NaN,NaN,NaN,NaN,2153.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
3,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187531074.0,15,D3317N,C>T,c.9949G>A,NM_005245.4,50.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV71675068,rs199989457,Likely_benign,709287.0,NaN,NaN,4731.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
4,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,ABRAXAS1,4,84383761.0,9,D364G,T>C,c.1091A>G,NM_139076.3,49.66,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV55029548,rs142910445,NaN,NaN,NaN,NaN,4748.0,18.93,NaN,0.584428,Detected,0.34006,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable


In [48]:
# Remove duplication
df_VHI01_IST01_01batch_dedup = df_VHI01_IST01_01batch.drop_duplicates(subset = "GHSampleID")
df_VHI01_IST01_01batch_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable
90,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable
91,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,14.66,NaN,0.137291,Detected,0.369632,NaN,Not evaluable,Not Detected,74.88,2.5,0,11/9/23,9/9/20,12/6/24,NaN,Not Applicable,Not Applicable
210,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.436673,Detected,0.990000,NaN,Not evaluable,Not Detected,474.24,3.5,0,11/9/23,1/25/21,12/6/24,NaN,Not Applicable,Not Applicable
454,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.931165,Detected,0.976160,20.80,High,Not Detected,481.52,3.0,0,11/9/23,5/4/21,12/6/24,NaN,Not Applicable,Not Applicable


In [49]:
# readin the intermediate files with correct "fc_dir" information including from rerun folder or Sirius 1.6.1 folder
df_VHI01_IST01_01batch_dedup_2_batch3 = pd.read_csv(f"{work_dir}/df_VHI01_IST01_01batch_dedup_2_batch3_118_samples_with_fc_dir_02202025.csv")
df_VHI01_IST01_01batch_dedup_2_batch1and2 = pd.read_csv(f"{work_dir}/df_VHI01_IST01_01batch_dedup_2_batch1and2_147_samples_with_rerun_fc_dir_02202025.csv")

df_runid_fcdir = pd.concat([df_VHI01_IST01_01batch_dedup_2_batch3, df_VHI01_IST01_01batch_dedup_2_batch1and2])
print(df_runid_fcdir.shape)
df_runid_fcdir.head()

(265, 58)


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
0,mCRPC_BIOBANK,GU3871,A1273171,B00911075,PRO349,Baseline,1,SUCCESS,NaN,SNV,NaN,ARID2,12,46246471.0,15.0,T1522S,C>G,c.4565C>G,NM_152641.4,52.36,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV57602507,rs370721871,NaN,NaN,NaN,NaN,5199.0,4.02,NaN,0.088887,Detected,0.036036,8.54,Low,Not Detected,31.24,2.5,0.0,01/16/2025,04/22/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF,/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius
1,mCRPC_BIOBANK,GU3111,A1273172,B00909952,PRO207,Progression,1,SUCCESS,NaN,SNV,NaN,TYMP,22,50964895.0,7.0,L313L,C>T,c.939G>A,NM_001113756.3,55.29,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1414.0,0.51,NaN,0.019688,Detected,0.004177,NaN,Not evaluable,Not Detected,29.20,2.5,0.0,01/16/2025,06/13/2023,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250125_A00277_1224_BH25KGDSXF,/ghsfa/ivd/flowcentral/250125_A00277_1224_BH25KGDSXF.cbb750c2-c173-4d59-9da5-d4e4b3be566f.20250127142438/sirius
2,mCRPC_BIOBANK,GU2264,A1273173,B00912113,PRO166,Progression,1,SUCCESS,NaN,SNV,NaN,MERTK,2,112705105.0,4.0,E240K,G>A,c.718G>A,NM_006343.3,52.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV54927603,rs868703580,NaN,NaN,NaN,NaN,4370.0,8.19,NaN,0.022271,Detected,0.717642,NaN,Not evaluable,Not Detected,49.13,1.5,0.0,01/16/2025,06/09/2022,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF,/ghsfa/ivd/flowcentral/250126_A00770_1124_BH25LJDSXF.02cabc46-d2d7-4199-b668-1794759155c3.20250128114740/sirius
3,mCRPC_BIOBANK,GU1055,A1273174,B00911090,PRO016,Baseline,1,SUCCESS,NaN,SNV,NaN,MAP3K13,3,185198208.0,13.0,I897T,T>C,c.2690T>C,NM_004721.5,51.47,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53987036,rs372763568,NaN,NaN,NaN,NaN,4964.0,4.87,NaN,0.009195,Detected,0.088850,7.59,Low,Not Detected,24.82,2.5,0.0,01/16/2025,07/10/2020,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A01422_0773_AH272YDSXF,/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius
4,mCRPC_BIOBANK,GU3752,A1273175,B00912118,PRO167,Progression,1,SUCCESS,NaN,SNV,NaN,REV3L,6,111696540.0,14.0,L1006L,T>C,c.3018A>G,NM_002912.5,52.51,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs767320584,NaN,NaN,NaN,NaN,2370.0,10.35,NaN,0.022506,Detected,0.010723,7.59,Low,Not Detected,20.59,3.0,0.0,01/16/2025,03/04/2024,02/07/2025,Metastatic prostate cancer,Not Applicable,Not Applicable,250126_A00770_1124_BH25LJDSXF,/ghsfa/ivd/flowcentral/250126_A00770_1124_BH25LJDSXF.02cabc46-d2d7-4199-b668-1794759155c3.20250128114740/sirius


In [50]:
# Merge the "runid" and "fc_dir" back
df_VHI01_IST01_01batch_runid.rename(columns={'run_sample_id': 'GHSampleID'}, inplace=True)
df_VHI01_IST01_01batch_dedup_2 = df_VHI01_IST01_01batch_dedup.merge(df_runid_fcdir[["GHSampleID", "runid", 'fc_dir']], on = "GHSampleID", how = "left")
print("Total unique sample size of df_VHI01_IST01 Infinity batch with missing fc_dir is " + str(df_VHI01_IST01_01batch_dedup_2.fc_dir.isnull().sum()) + "\n")
df_VHI01_IST01_01batch_dedup_2.head()

Total unique sample size of df_VHI01_IST01 Infinity batch with missing fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0
1,mCRPC_BIOBANK,GU1770,A0914041,B00586988,PRO191,Progression,0,FAIL,Low Diversity; Methylation Bias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.50,2.5,0,11/9/23,10/21/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0
2,mCRPC_BIOBANK,GU1133,A0914042,B00586989,PRO098,Baseline,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,57.19,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,2564.0,14.66,NaN,0.137291,Detected,0.369632,NaN,Not evaluable,Not Detected,74.88,2.5,0,11/9/23,9/9/20,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0
3,mCRPC_BIOBANK,GU1332,A0914043,B00586986,PRO098,Progression,1,SUCCESS,NaN,SNV,NaN,ATM,11,108199845.0,49,T2396S,C>G,c.7187C>G,NM_000051.4,88.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV53755657,rs370559102,Conflicting_interpretations_of_pathogenicity,132895.0,NaN,NaN,4123.0,41.04,NaN,0.436673,Detected,0.990000,NaN,Not evaluable,Not Detected,474.24,3.5,0,11/9/23,1/25/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0
4,mCRPC_BIOBANK,GU1492,A0914044,B00586987,PRO153,Progression,1,SUCCESS,NaN,SNV,NaN,TFE3,X,48887989.0,10,G470S,C>T,c.1408G>A,NM_006521.6,100.00,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs147974460,NaN,NaN,NaN,NaN,2095.0,24.26,NaN,0.931165,Detected,0.976160,20.80,High,Not Detected,481.52,3.0,0,11/9/23,5/4/21,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0


In [51]:
df_VHI01_IST01_01batch_dedup_2.fc_dir.value_counts()

/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240921_A01744_0277_BHKVGCDSXC.Sirius-1.6.0/sirius_1_6_0    30
/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240921_A02019_0190_AHKNVNDSXC.Sirius-1.6.0/sirius_1_6_0    30
/ghsfa/ivd/flowcentral/250126_A01422_0773_AH272YDSXF.2c6bd3ff-3748-4a26-baf4-b3599ad72fe3.20250128162437/sirius                                   30
/ghsfa/ivd/flowcentral/250125_A01613_0564_BH27NGDSXF.b3ef1d3c-fcd3-4084-a167-1bad62072639.20250127134830/sirius                                   30
/ghsfa/ivd/flowcentral/250126_A01422_0774_BH25NGDSXF.27b16eb4-4124-402d-8ebf-2a54f5e3ff11.20250128121631/sirius                                   30
/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240921_A02019_0189_BHKTTCDSXC.Sirius-1.6.0/sirius_1_6_0    29
/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_

## Generate Pancancer S3 MB caller and Single region caller outputs for this SOW
#### Notes:
    1. No cancer type provided
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update

In [52]:
# Copying the dataframe
df_VHI01_IST01_01batch_dedup_3 = df_VHI01_IST01_01batch_dedup_2.copy()

# List of target genes
Target_PM_gene_list = ["APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", 
                       "CDK12", "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", 
                       "FANCL", "KEAP1", "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", 
                       "PALB2", "PBRM1", "PMS2", "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", 
                       "RB1", "SDHA", "SDHB", "SDHC", "SDHD", "SMAD4", "STK11", "TP53", "TSC2"]

# Initialize columns for each gene in the dataframe
for gene in Target_PM_gene_list:
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call"] = np.nan
    df_VHI01_IST01_01batch_dedup_3[f"{gene}_single_region_call_mol"] = np.nan

problematic_rows = []  # List to store indices of problematic rows

# Loop through the rows of the dataframe
for index, crc_row in df_VHI01_IST01_01batch_dedup_3.iterrows():
    fc_dir = crc_row["fc_dir"]
    run_sample_id = crc_row["GHSampleID"]

    # Define the file path for the single region report
    single_region_file = f"{fc_dir}/{run_sample_id}/{run_sample_id}.single_region_call.report.hdr.tsv"

    try:
        # Read the single region call file
        single_df = pd.read_table(single_region_file)

        if not single_df.empty:
            # Loop through the target gene list and assign the call values to the corresponding columns
            for gene in Target_PM_gene_list:
                gene_df = single_df[single_df['gene'] == gene]
                if not gene_df.empty:
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call"] = gene_df.iloc[0]['call']
                    df_VHI01_IST01_01batch_dedup_3.at[index, f"{gene}_single_region_call_mol"] = gene_df.iloc[0]['molecules']
        else:
            # Add the index to the list if the dataframe is empty
            problematic_rows.append(index) 

    except Exception as e:
        # Add the index to the list if there is any exception
        problematic_rows.append(index)  
        print(f"Error processing file {single_region_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_VHI01_IST01_01batch_dedup_3.loc[problematic_rows]

print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is {df_problematic.shape[0]}\n")

# Show the updated dataframe with the new columns
df_VHI01_IST01_01batch_dedup_3.head()


Total sample size of VHI01_IST01_Infinity Infinity batch with NO output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,APC_single_region_call,APC_single_region_call_mol,ARID1A_single_region_call,ARID1A_single_region_call_mol,ARID1B_single_region_call,ARID1B_single_region_call_mol,ATM_single_region_call,ATM_single_region_call_mol,ATR_single_region_call,ATR_single_region_call_mol,B2M_single_region_call,B2M_single_region_call_mol,BRCA1_single_region_call,BRCA1_single_region_call_mol,BRCA2_single_region_call,BRCA2_single_region_call_mol,BRIP1_single_region_call,BRIP1_single_region_call_mol,CDH1_single_region_call,CDH1_single_region_call_mol,CDK12_single_region_call,CDK12_single_region_call_mol,CDKN1B_single_region_call,CDKN1B_single_region_call_mol,CDKN2A_single_region_call,CDKN2A_single_region_call_mol,CHEK1_single_region_call,CHEK1_single_region_call_mol,CHEK2_single_region_call,CHEK2_single_region_call_mol,FANCA_single_region_call,FANCA_single_region_call_mol,FANCE_single_region_call,FANCE_single_region_call_mol,FANCF_single_region_call,FANCF_single_region_call_mol,FANCG_single_region_call,FANCG_single_region_call_mol,FANCL_single_region_call,FANCL_single_region_call_mol,KEAP1_single_region_call,KEAP1_single_region_call_mol,MGMT_single_region_call,MGMT_single_region_call_mol,MLH1_single_region_call,MLH1_single_region_call_mol,MSH2_single_region_call,MSH2_single_region_call_mol,MSH6_single_region_call,MSH6_single_region_call_mol,MTAP_single_region_call,MTAP_single_region_call_mol,MUTYH_single_region_call,MUTYH_single_region_call_mol,NF1_single_region_call,NF1_single_region_call_mol,NF2_single_region_call,NF2_single_region_call_mol,PALB2_single_region_call,PALB2_single_region_call_mol,PBRM1_single_region_call,PBRM1_single_region_call_mol,PMS2_single_region_call,PMS2_single_region_call_mol,POLE_single_region_call,POLE_single_region_call_mol,PTEN_single_region_call,PTEN_single_region_call_mol,RAD50_single_region_call,RAD50_single_region_call_mol,RAD51_single_region_call,RAD51_single_region_call_mol,RAD51C_single_region_call,RAD51C_single_region_call_mol,RAD51D_single_region_call,RAD51D_single_region_call_mol,RB1_single_region_call,RB1_single_region_call_mol,SDHA_single_region_call,SDHA_single_region_call_mol,SDHB_single_region_call,SDHB_single_region_call_mol,SDHC_single_region_call,SDHC_single_region_call_mol,SDHD_single_region_call,SDHD_single_region_call_mol,SMAD4_single_region_call,SMAD4_single_region_call_mol,STK11_single_region_call,STK11_single_region_call_mol,TP53_single_region_call,TP53_single_region_call_mol,TSC2_single_region_call,TSC2_single_region_call_mol
0,mCRPC_BIOBANK,GU0680,A0914040,B00587013,PRO074,Baseline,1,SUCCESS,NaN,SNV,NaN,FGFR3,4,1806189.0,9,K403R,A>G,c.1208A>G,NM_000142.5,52.45,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772301946,NaN,NaN,NaN,NaN,5768.0,18.93,NaN,0.584428,Detected,0.340060,9.45,Low,Not Detected,237.12,2.5,0,11/9/23,10/15/19,12/6/24,NaN,Not Applicable,Not Applicable,240125_A01361_0592_AHMMTVDSX7,/ghds/projects/pharma/rerun_flowcentral/BART-1314_VHI01_IST01_Infinity_rerun/flowcells/240125_A01361_0592_AHMMTVDSX7.Sirius-1.6.0/sirius_1_6_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [54]:
# Check any edge case for reportable PM when tumor methylation status is dectected
# Promoter_methylation_count > 5 & Alt_NoAlt == 1 & Max_percentage > 1%
# This is the confluence page: https://guardanthealth.atlassian.net/wiki/spaces/PHARMA/pages/3027927678/Part+II.+Automated+review+by+the+pipeline 

df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3[(df_VHI01_IST01_01batch_dedup_3['Tumor_methylation_status'] == 'Not Detected') & (df_VHI01_IST01_01batch_dedup_3['Alt-NoAlt'] == 1) & (df_VHI01_IST01_01batch_dedup_3['Max_percentage'] > 1)]
# Select only columns that match the pattern '{gene}_single_region_call_mol'
numeric_columns = [col for col in df_VHI01_IST01_01batch_dedup_3_edge.columns if col.endswith("_single_region_call_mol")]

# Apply filter: Keep rows where any of these numeric columns have a value greater than 5
df_VHI01_IST01_01batch_dedup_3_edge = df_VHI01_IST01_01batch_dedup_3_edge[(df_VHI01_IST01_01batch_dedup_3_edge[numeric_columns] > 5).any(axis=1)]
print(f"Total sample size of VHI01_IST01_Infinity Infinity batch with potential reportable PM edge cases is {df_VHI01_IST01_01batch_dedup_3_edge.shape[0]}\n")

# Print out the edge PM cases
filtered_results = []

# Iterate over each row that meets the condition
for index, row in df_VHI01_IST01_01batch_dedup_3_edge.iterrows():
    GHSampleID = row["GHSampleID"]  # Extract the sample ID
    for col in numeric_columns:
        if row[col] > 5:  # Check if value is greater than 5
            # Derive the corresponding "_single_region_call" column name
            gene_name = col.replace("_single_region_call_mol", "")
            single_region_call_col = f"{gene_name}_single_region_call"

            # Check if this column exists in the DataFrame before retrieving its value
            single_region_call_value = row[single_region_call_col] if single_region_call_col in row else "N/A"

            # Append to results
            filtered_results.append({
                "GHSampleID": GHSampleID,
                "Column": col,
                "Value": row[col],
                "Single_Region_Call_Column": single_region_call_col,
                "Single_Region_Call_Value": single_region_call_value
            })

# Convert results to DataFrame for easy viewing
df_filtered_results = pd.DataFrame(filtered_results)
df_filtered_results

Total sample size of VHI01_IST01_Infinity Infinity batch with potential reportable PM edge cases is 4



,GHSampleID,Column,Value,Single_Region_Call_Column,Single_Region_Call_Value
0,B00911088,CDH1_single_region_call_mol,11.0,CDH1_single_region_call,0.0
1,B00912115,BRIP1_single_region_call_mol,10.0,BRIP1_single_region_call,0.0
2,B00911114,CDH1_single_region_call_mol,6.0,CDH1_single_region_call,0.0
3,B00910016,CDH1_single_region_call_mol,7.0,CDH1_single_region_call,0.0
4,B00910016,PMS2_single_region_call_mol,7.0,PMS2_single_region_call,0.0
5,B00910016,RAD51_single_region_call_mol,6.0,RAD51_single_region_call,0.0


In [ ]:
# Based on above cell result, none of the potential reportable PM in methylation status not detected case are realy reportbale!

In [55]:
# List to store filtered results
filtered_results = []

# Iterate over each row that meets the condition
for index, row in df_VHI01_IST01_01batch_dedup_3_edge.iterrows():
    GHSampleID = row["GHSampleID"]  # Extract the sample ID
    for col in numeric_columns:
        if row[col] > 5:  # Check if value is greater than 5
            # Derive the corresponding "_single_region_call" column name
            gene_name = col.replace("_single_region_call_mol", "")
            single_region_call_col = f"{gene_name}_single_region_call"

            # Check if this column exists in the DataFrame before retrieving its value
            single_region_call_value = row[single_region_call_col] if single_region_call_col in row else "N/A"

            # Append to results
            filtered_results.append({
                "GHSampleID": GHSampleID,
                "Column": col,
                "Value": row[col],
                "Single_Region_Call_Column": single_region_call_col,
                "Single_Region_Call_Value": single_region_call_value
            })

# Convert results to DataFrame for easy viewing
df_filtered_results = pd.DataFrame(filtered_results)

df_filtered_results

,GHSampleID,Column,Value,Single_Region_Call_Column,Single_Region_Call_Value
0,B00911088,CDH1_single_region_call_mol,11.0,CDH1_single_region_call,0.0
1,B00912115,BRIP1_single_region_call_mol,10.0,BRIP1_single_region_call,0.0
2,B00911114,CDH1_single_region_call_mol,6.0,CDH1_single_region_call,0.0
3,B00910016,CDH1_single_region_call_mol,7.0,CDH1_single_region_call,0.0
4,B00910016,PMS2_single_region_call_mol,7.0,PMS2_single_region_call,0.0
5,B00910016,RAD51_single_region_call_mol,6.0,RAD51_single_region_call,0.0


In [56]:
# Retrive the 47 PM gene binary call result

df_VHI01_IST01_01batch_dedup_4 = df_VHI01_IST01_01batch_dedup_3.copy()

# List of 47 LDT reportable PM genes
Target_PM_gene_list = [
    "APC", "ARID1A", "ARID1B", "ATM", "ATR", "B2M", "BRCA1", "BRCA2", "BRIP1", "CDH1", "CDK12", 
    "CDKN1B", "CDKN2A", "CHEK1", "CHEK2", "FANCA", "FANCE", "FANCF", "FANCG", "FANCL", "KEAP1", 
    "MGMT", "MLH1", "MSH2", "MSH6", "MTAP", "MUTYH", "NF1", "NF2", "PALB2", "PBRM1", "PMS2", 
    "POLE", "PTEN", "RAD50", "RAD51", "RAD51C", "RAD51D", "RB1", "SDHA", "SDHB", "SDHC", 
    "SDHD", "SMAD4", "STK11", "TP53", "TSC2"
]

# Process each gene for each sample
new_rows = []
for index, row in df_VHI01_IST01_01batch_dedup_4.iterrows():
    GHSampleID = row['GHSampleID']
    for gene in Target_PM_gene_list:
        gene_call = row[f'{gene}_single_region_call']
        if pd.notna(gene_call):
            variant_type = 'PromoterMethylationDetected' if gene_call == 1 else 'PromoterMethylationNotDetected'
            new_rows.append({'GHSampleID': GHSampleID, 'Variant_type': variant_type, 'Gene': gene})

# Append new rows to the dataframe
df_47_PM_gene = pd.DataFrame(new_rows)
print(df_47_PM_gene.shape)
df_47_PM_gene.head()


(12455, 3)


,GHSampleID,Variant_type,Gene
0,B00587013,PromoterMethylationNotDetected,APC
1,B00587013,PromoterMethylationNotDetected,ARID1A
2,B00587013,PromoterMethylationNotDetected,ARID1B
3,B00587013,PromoterMethylationNotDetected,ATM
4,B00587013,PromoterMethylationNotDetected,ATR


In [57]:
# Remove the "PromoterMethylation" rows and replace the new 47 PM results into each GHSampleID

# Step 1: Separate the rows where 'Variant_type' is 'PromoterMethylation'
df_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] == 'PromoterMethylation'].copy()

# Step 2: Remove these rows from the original dataframe to avoid duplication later
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()

# Step 3: Include the necessary columns
df_promoter_methylation_updated = df_promoter_methylation[['Study_ID', 'Customer_SampleID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
                                                           'Sample_status', 'Sample_comment', 'Max_percentage', 'Alleletype', 'HRD_score', 
                                                           'Tumor_methylation_status', 'Tumor_methylation_score', 'TMB_score', 'TMB_category', 
                                                           'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 'Received_date', 
                                                           'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name',
                                                           'Variant_type', 'Gene']].copy()

# Step 4: Filter out invalid GHSampleIDs based on Sample_status and Tumor_methylation_status
# Based on SOP, Not Detected case are more likely to suppress all the PM calls
valid_ghsampleid = df_VHI01_IST01_01batch[
    (df_VHI01_IST01_01batch['Sample_status'] != 'FAIL') & 
    (df_VHI01_IST01_01batch['Tumor_methylation_status'] != 'Not Detected')
]['GHSampleID'].unique()

# Retain only rows with valid GHSampleIDs in df_promoter_methylation_updated
df_promoter_methylation_updated = df_promoter_methylation_updated[
    df_promoter_methylation_updated['GHSampleID'].isin(valid_ghsampleid)
]

# Step 5: Filter df_47_PM_gene to only include valid GHSampleIDs
df_47_PM_gene_filtered = df_47_PM_gene[df_47_PM_gene['GHSampleID'].isin(valid_ghsampleid)].copy()

# Step 6: Merge with the filtered df_47_PM_gene using a left join
df_promoter_methylation_updated = df_promoter_methylation_updated.merge(
    df_47_PM_gene_filtered, on='GHSampleID', how='left', suffixes=('', '_PM')
)

# Step 7: Replace 'Variant_type' and 'Gene' with the new values from df_47_PM_gene
df_promoter_methylation_updated['Variant_type'] = df_promoter_methylation_updated['Variant_type_PM']
df_promoter_methylation_updated['Gene'] = df_promoter_methylation_updated['Gene_PM']

# Step 8: Drop the temporary columns generated during the merge
df_promoter_methylation_updated.drop(columns=['Variant_type_PM', 'Gene_PM'], inplace=True)

# Step 9: Identify GHSampleIDs that do not have any "PromoterMethylation" rows
ghsampleid_with_pm_rows = df_promoter_methylation['GHSampleID'].unique()
all_ghsampleid = df_VHI01_IST01_01batch['GHSampleID'].unique()
missing_ghsampleid = set(all_ghsampleid) - set(ghsampleid_with_pm_rows)

# Filter missing GHSampleIDs to only include valid ones
missing_ghsampleid = missing_ghsampleid.intersection(valid_ghsampleid)

# Step 10: Create the "PromoterMethylation" rows for the filtered GHSampleIDs
df_missing_pm_rows = df_47_PM_gene_filtered[df_47_PM_gene_filtered['GHSampleID'].isin(missing_ghsampleid)].copy()

# Populate the necessary columns by retrieving values for the same GHSampleID
columns_to_populate = [
    'Study_ID', 'Customer_SampleID', 'GHRequestID', 'Patient_ID', 'Visit_name', 'Alt-NoAlt', 
    'Sample_status', 'Sample_comment', 'Max_percentage', 'Tumor_methylation_status', 'Tumor_methylation_score', 
    'TMB_score', 'TMB_category', 'MSI_High', 'cfDNA_ng', 'Plasma_ml_input', 'Plasma_ml_remaining', 
    'Received_date', 'Bloodcoll_date', 'Reported_date', 'Cancertype', 'Practice_name', 'Physician_name'
]

for col in columns_to_populate:
    if col in df_VHI01_IST01_01batch.columns:
        df_missing_pm_rows[col] = df_missing_pm_rows['GHSampleID'].map(
            df_VHI01_IST01_01batch.groupby('GHSampleID')[col].first()
        )
    else:
        print(f"Column {col} not found in source dataframe, leaving as NaN")

# Step 11: Append the missing rows to the updated promoter methylation rows
df_promoter_methylation_updated = pd.concat([df_promoter_methylation_updated, df_missing_pm_rows], ignore_index=True)

# Step 12: Append the updated rows back to the original dataframe while keeping their positions
df_ori_with_47pm = pd.concat([df_non_promoter_methylation, df_promoter_methylation_updated], ignore_index=True)

# Step 13: Sort the dataframe by GHSampleID and place rows accordingly
df_ori_with_47pm.sort_values(by=['GHSampleID', 'Visit_name'], inplace=True)

# Step 14: Ensure there are no duplicates
df_ori_with_47pm.drop_duplicates(inplace=True)

# Display the updated dataframe
df_ori_with_47pm.head()


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
655,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,MYC,8,128750848.0,2,S129G,A>G,c.385A>G,NM_002467.6,62.03,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6105.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
656,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,TET2,4,106156445.0,3,V449A,T>C,c.1346T>C,NM_001127208.3,52.75,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs753698541,NaN,NaN,NaN,NaN,6245.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
657,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FAAP100,17,79511105.0,7,A784A,G>A,c.2352C>T,NM_025161.6,51.45,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs142183217,NaN,NaN,NaN,NaN,5688.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
658,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,LRP5,11,68177472.0,10,M728L,A>T,c.2182A>T,NM_002335.4,51.09,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs149241739,NaN,NaN,NaN,NaN,5526.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable
659,mCRPC_BIOBANK,GU0646,A0914047,B00586981,PRO030,Progression,1,SUCCESS,NaN,SNV,NaN,FOXA1,14,38060693.0,2,T432T,C>G,c.1296G>C,NM_004496.5,51.06,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs375938424,NaN,NaN,NaN,NaN,5745.0,20.41,NaN,0.028785,Detected,0.407858,15.12,Low,Not Detected,83.2,2.5,0,11/9/23,9/23/19,12/6/24,NaN,Not Applicable,Not Applicable


In [58]:
# Check failed cases
df_ori_with_47pm_fail = df_ori_with_47pm[df_ori_with_47pm['GHSampleID'] == 'B00911129']
df_ori_with_47pm_fail

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
11257,mCRPC_BIOBANK,GU4214/ESR,A1273198,B00911129,PRO359,Baseline,0,FAIL,"Methylation Fail, Low Diversity",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.54,2.0,0,1/16/25,8/12/24,2/7/25,Metastatic prostate cancer,Not Applicable,Not Applicable


In [59]:
# Check the result 
print(df_VHI01_IST01_01batch.shape)
print(df_47_PM_gene.shape)
df_non_promoter_methylation = df_VHI01_IST01_01batch[df_VHI01_IST01_01batch['Variant_type'] != 'PromoterMethylation'].copy()
print(df_non_promoter_methylation.shape)
print(df_ori_with_47pm.shape)

# Save the temporary data
df_ori_with_47pm.to_csv(f"{work_dir}/df_ori_with_47pm_batch_all_02192025.csv", index = False)

(19298, 56)
(12455, 3)
(16228, 56)
(27743, 56)


In [60]:
# Additional dataframe updates based on Rachael's request
# Drop two columns
df_ori_with_47pm_removed_2col = df_ori_with_47pm.drop(columns=['HRD_score', 'Alleletype'])

# Drop IO Allele type info
df_final = df_ori_with_47pm_removed_2col[df_ori_with_47pm_removed_2col['Variant_type'] != "AlleleType"]

# Save the data
df_final.to_csv(f"{work_dir}/df_final_batch_all_02192025.csv", index = False)
df_final.shape

(27743, 54)